In [202]:
from functools import partial
from pathlib import Path
from typing import List, Dict, Callable

import pandas as pd
import numpy as np
import json

In [204]:
from functools import wraps, WRAPPER_ASSIGNMENTS

try:
    wraps(partial(wraps))(wraps)
except AttributeError:
    @wraps(wraps)
    def wraps(obj, attr_names=WRAPPER_ASSIGNMENTS, wraps=wraps):
        return wraps(obj, assigned=(name for name in attr_names if hasattr(obj, name)))

In [3]:
def get_total_score(x: str) -> float:
    js = json.loads(x)
    size = len(js.items())

    return sum([js[f]['drift_detected'] for f,v in js.items()]) / size

def get_regress_score(x: str) -> float:
    js = json.loads(x)

    # TODO: use different aggregation methods
    return sum([js[f]['drift_score'] for f,v in js.items()])

In [78]:
def get_weighted_aggregation(x: str, fi: Dict, method: Callable = sum):
    js = json.loads(x)

    return method([js[f]['drift_score'] * fi.get(f, 1) for f,v in js.items()])


In [87]:
def get_weighted_thresh_agg(x: str, fi: Dict, method: Callable = sum):
    js = json.loads(x)

    return method([js[f]['drift_score'] * fi.get(f, 1) // js[f]['stattest_threshold'] for f,v in js.items()])


In [220]:
fi = pd.read_csv('data/generated/fi_2020-2023.csv', index_col=0)
fi.rename(index={'Lower_limit': 'Param 1', 'Upper_limit': 'Param 2'}, inplace=True)
fi = fi / fi.max()
fi = fi.to_dict()['importance']
fi

{'HR Usage Rate': 1.0,
 'TCH Blocking Rate, BH': 0.05306688095919128,
 'Number of Available\nTCH': 0.9145249190288962,
 'TCH Traffic (Erl), BH': 0.48950080102867455,
 'Param 1': 0.934843836886342,
 'Param 2': 0.8525678849265464}

In [221]:
get_weighted_thresh_sum = partial(get_weighted_thresh_agg, fi={}, method=np.average)
get_weighted_thresh_sum.__name__ = 'get_weighted_thresh_sum'

get_thresh_sum = partial(get_weighted_thresh_agg, fi={}, method=np.average)
get_thresh_sum.__name__ = 'get_thresh_sum'

get_weighted_mean = partial(get_weighted_aggregation, fi=fi, method=np.sum)
get_weighted_mean.__name__ = 'get_weighted_mean'

In [216]:
get_weighted_thresh_sum.__name__

'get_weighted_thresh_sum'

In [4]:
rewards = pd.read_csv('data/generated/drift/by_cell_agent/drift_scores_rewards_new_agent_train-test_no_sample20-23.csv', index_col=0)
rewards

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,mom_reward_min,mom_reward_max,mom_reward_std
0,10932,0.875,0.991378,0.973693,0.994549,0.002269,1040.000000,2070.0,598.999165,10.000000,10.0,10.0,0.000000
1,12762,0.875,0.986723,0.790479,0.992018,0.019664,1010.000000,2040.0,598.999165,9.855072,-20.0,10.0,2.085144
2,12781,0.875,0.987593,0.949088,0.992366,0.004489,1040.000000,2070.0,598.999165,10.000000,10.0,10.0,0.000000
3,12782,0.875,0.986304,0.972973,0.992206,0.003253,1040.000000,2070.0,598.999165,10.000000,10.0,10.0,0.000000
4,12783,1.000,0.949899,0.779899,0.991859,0.049240,925.797101,1920.0,558.289010,9.275362,-20.0,10.0,4.617035
...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,5683,0.875,0.992266,0.967622,0.993875,0.002159,800.000000,1590.0,460.434577,10.000000,10.0,10.0,0.000000
990,5682,0.875,0.991784,0.980978,0.994072,0.001335,800.000000,1590.0,460.434577,10.000000,10.0,10.0,0.000000
991,12771,0.875,0.990560,0.978193,0.994569,0.002849,570.000000,1130.0,327.643099,10.000000,10.0,10.0,0.000000
992,12772,1.000,0.968540,0.751103,0.986172,0.028634,550.884956,1100.0,315.678288,9.734513,-20.0,10.0,2.822163


In [154]:
rewards.describe()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,mom_reward_min,mom_reward_max,mom_reward_std
count,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.0,994.000000
mean,14601.775654,0.909079,0.985404,0.874050,0.993143,0.015381,982.392422,1989.825568,584.887036,9.735159,-3.908601,10.0,1.624021
std,11994.634963,0.055692,0.010261,0.141991,0.001913,0.020892,144.251080,182.222544,35.048051,0.850586,28.322576,0.0,3.758170
min,701.000000,0.875000,0.874052,0.379827,0.969809,0.000385,-765.904569,-152.168731,283.425739,-3.487866,-172.640971,10.0,0.000000
25%,5071.250000,0.875000,0.983476,0.820343,0.992224,0.002218,965.126812,1980.000000,588.948638,9.855072,-20.000000,10.0,0.000000
50%,10931.500000,0.875000,0.988649,0.938276,0.993460,0.006363,1040.000000,2070.000000,598.999165,10.000000,10.000000,10.0,0.000000
75%,24234.500000,1.000000,0.991324,0.976942,0.994365,0.019126,1040.000000,2070.000000,598.999165,10.000000,10.000000,10.0,2.085144
max,42857.000000,1.000000,0.993943,0.992224,0.996884,0.156021,1040.000000,2070.000000,598.999165,10.000000,10.000000,10.0,39.482828


# NOT Sampled | no ref window

In [156]:
orig_df = pd.read_csv('data/generated/drift/by_cell_agent/run_7/by_train_regressive__sampled-drift-None_no-window_.csv', index_col=0)
orig_df

,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test,cell_id
0,"{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",25771
1,"{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",22944
2,"{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",26335
3,"{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",26332
4,"{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",26336
...,...,...,...,...,...,...,...,...,...,...,...
989,"{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",5683
990,"{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ..

In [91]:
stat_tests = [c for c in orig_df.columns if c not in ['cell_id']]

drift_scores_df = orig_df[['cell_id']].copy()

drift_scores_df[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_total_score, x)))
drift_scores_df

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,1.0,0.833333,0.833333,0.833333
1,22944,0.833333,1.000000,0.833333,0.833333,0.833333,0.833333,1.0,0.833333,0.833333,0.833333
2,26335,1.000000,1.000000,1.000000,0.833333,0.833333,0.833333,1.0,0.833333,1.000000,0.666667
3,26332,1.000000,1.000000,1.000000,0.833333,0.833333,0.833333,1.0,0.833333,1.000000,0.833333
4,26336,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,1.0,0.833333,0.833333,0.833333
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,1.0,0.833333,0.833333,0.833333
990,5682,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,1.0,0.833333,0.833333,0.666667
991,12772,1.000000,1.000000,1.000000,0.833333,0.833333,0.833333,1.0,0.833333,1.000000,0.833333
992,12771,0.833333,1.000000,0.833333,0.833333,0.833333,0.833333,1.0,0.833333,0.833333,0.833333


## Regress

In [92]:
drift_regress_df = orig_df[['cell_id']].copy()

drift_regress_df[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_regress_score, x)))
drift_regress_df

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,4.624301,7.704001e-02,4.624301,22.958075,35.822121,3.235612,0.000232,3.521493,18.980594,0.492405
1,22944,8.513001,9.403760e-03,8.513001,27.952476,40.319391,3.606562,0.000232,3.914418,30.671887,0.227054
2,26335,4.499932,1.413220e-29,4.499932,21.171273,32.038256,3.149558,0.000232,3.409220,18.491783,0.834270
3,26332,4.831547,1.235722e-11,4.831547,22.517829,34.787749,3.141581,0.000232,3.376812,19.523089,0.133723
4,26336,5.083292,6.001640e-01,5.083292,24.718289,36.286952,3.366172,0.000232,3.548588,20.341524,0.349846
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,4.143555,1.532900e-01,4.143555,19.642791,27.837917,3.156555,0.000266,3.152360,16.366687,0.319351
990,5682,3.914246,7.217293e-02,3.914246,22.653205,31.603491,3.302213,0.000266,3.133916,14.968536,0.711898
991,12772,5.306576,1.792384e-23,5.306576,26.507881,38.908958,3.585255,0.000317,3.772797,21.300469,0.275323
992,12771,4.427045,3.724132e-02,4.427045,20.072339,27.145024,3.041050,0.000317,2.924274,17.377473,0.493994


In [93]:
merged = rewards.merge(drift_scores_df, left_on='cell_id', right_on='cell_id', how='inner')
merged.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,-0.042054,-0.014083,-0.042054,-0.004797,-0.029400,-0.035427,NaN,-0.027563,-0.032091,-0.013158
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,1.000000,0.348880,1.000000,0.116137,0.387639,0.391487,NaN,0.391487,0.781903,0.240570
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,-0.637122,-0.237932,-0.637122,-0.490077,-0.601974,-0.594971,NaN,-0.568248,-0.562093,-0.305755
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,-0.609597,-0.056428,-0.609597,-0.164067,-0.547614,-0.547616,NaN,-0.564737,-0.502250,-0.325514
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,-0.289934,-0.168453,-0.289934,-0.067899,-0.125177,-0.119252,NaN,-0.115056,-0.309426,-0.129502
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,0.642976,0.093760,0.642976,0.266944,0.676731,0.676923,NaN,0.680283,0.531415,0.357839
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,-0.395991,-0.090844,-0.395991,-0.337963,-0.650346,-0.641243,NaN,-0.646223,-0.318352,-0.293259
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,-0.362477,-0.079346,-0.362477,-0.357040,-0.622673,-0.614825,NaN,-0.607506,-0.291314,-0.269860
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,-0.190325,-0.045423,-0.190325,-0.389688,-0.401941,-0.398805,NaN,-0.366093,-0.154870,-0.135084
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,-0.412066,-0.100554,-0.412066,-0.373832,-0.685072,-0.677233,NaN,-0.669658,-0.335622,-0.292552


In [94]:
merged = rewards.merge(drift_regress_df, left_on='cell_id', right_on='cell_id', how='inner')
merged.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,0.014442,0.007068,0.014442,0.027588,-0.000698,0.006128,-0.011665,0.018939,0.013517,-0.003948
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,0.158257,-0.237117,0.158257,0.049209,0.090908,0.152672,-0.038867,0.103360,0.150610,-0.094650
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,-0.348823,0.176814,-0.348823,-0.159974,-0.198297,-0.290886,0.019512,-0.210599,-0.256514,0.192356
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,-0.148415,0.014657,-0.148415,0.028709,-0.028558,-0.098536,0.047560,-0.041563,-0.083931,0.142976
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,-0.461979,0.158858,-0.461979,-0.461866,-0.387815,-0.470497,-0.047029,-0.440383,-0.439793,0.029824
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,0.173948,-0.051675,0.173948,-0.033062,0.028340,0.099225,-0.034263,0.039923,0.082746,-0.195058
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,-0.214143,0.078251,-0.214143,-0.005278,-0.038105,-0.096873,-0.231242,-0.026977,-0.072071,0.209303
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,-0.208836,0.079990,-0.208836,-0.013947,-0.042822,-0.097500,-0.421556,-0.027307,-0.073035,0.188320
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,-0.155785,0.068684,-0.155785,-0.039593,-0.049005,-0.082153,-0.700119,-0.034965,-0.065224,0.087185
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,-0.226402,0.080770,-0.226402,0.012223,-0.033538,-0.084069,0.027595,-0.028518,-0.074284,0.210355


In [95]:
merged.corr(method='kendall')

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.038715,0.008395,0.019957,-0.010198,-0.026838,0.006847,0.003968,0.005609,0.001734,...,0.013987,0.021193,0.013987,0.031920,0.014575,0.017144,-0.021954,0.026135,0.013720,-0.001759
drift_score,-0.038715,1.000000,-0.563385,-0.464556,-0.232765,0.505011,-0.378430,-0.367684,-0.307606,-0.499002,...,0.138535,-0.437358,0.138535,0.050112,0.082792,0.134927,-0.140509,0.094343,0.118519,-0.118136
quality_avg,0.008395,-0.563385,1.000000,0.590028,0.454029,-0.632093,0.419421,0.412733,0.357276,0.558258,...,-0.212893,0.257017,-0.212893,-0.143858,-0.163837,-0.217906,0.097356,-0.187438,-0.176250,-0.044538
quality_min,0.019957,-0.464556,0.590028,1.000000,0.196885,-0.884325,0.487317,0.477523,0.396062,0.637737,...,-0.097141,0.087079,-0.097141,0.010036,-0.026850,-0.066597,0.070491,-0.045409,-0.071310,-0.115628
quality_max,-0.010198,-0.232765,0.454029,0.196885,1.000000,-0.193189,0.154320,0.152633,0.140335,0.162721,...,-0.271054,0.172318,-0.271054,-0.284602,-0.235210,-0.290908,0.016500,-0.280788,-0.244956,-0.014306
quality_std,-0.026838,0.505011,-0.632093,-0.884325,-0.193189,1.000000,-0.490762,-0.482335,-0.397162,-0.648789,...,0.097270,-0.119817,0.097270,-0.011689,0.029241,0.068664,-0.075063,0.046195,0.070978,0.129415
cum_reward_avg,0.006847,-0.378430,0.419421,0.487317,0.154320,-0.490762,1.000000,0.963841,0.834348,0.731017,...,-0.135627,0.097083,-0.135627,-0.034913,-0.051853,-0.092185,-0.202111,-0.048970,-0.073996,0.110666
cum_reward_max,0.003968,-0.367684,0.412733,0.477523,0.152633,-0.482335,0.963841,1.000000,0.877907,0.728345,...,-0.137731,0.094741,-0.137731,-0.038212,-0.053999,-0.092615,-0.218052,-0.050110,-0.075381,0.105208
cum_reward_std,0.005609,-0.307606,0.357276,0.396062,0.140335,-0.397162,0.834348,0.877907,1.000000,0.587692,...,-0.132076,0.086931,-0.132076,-0.042892,-0.052562,-0.084390,-0.249198,-0.050183,-0.075790,0.079753
mom_reward_avg,0.001734,-0.499002,0.558258,0.637737,0.162721,-0.648789,0.731017,0.728345,0.587692,1.000000,...,-0.140700,0.135044,-0.140700,-0.002316,-0.038080,-0.086152,0.071465,-0.055216,-0.083461,0.135182


In [96]:
merged.corr(method='spearman')

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.047393,0.013536,0.029436,-0.014707,-0.039298,0.008819,0.004996,0.007599,0.001519,...,0.020643,0.031460,0.020643,0.046450,0.021518,0.024563,-0.034264,0.039617,0.020559,-0.004930
drift_score,-0.047393,1.000000,-0.689656,-0.568676,-0.284935,0.618199,-0.416448,-0.402147,-0.336589,-0.532794,...,0.169585,-0.534137,0.169585,0.061344,0.101348,0.165168,-0.172002,0.115488,0.145083,-0.144611
quality_avg,0.013536,-0.689656,1.000000,0.785649,0.628587,-0.823734,0.538676,0.525389,0.463892,0.687860,...,-0.330181,0.388712,-0.330181,-0.221744,-0.248162,-0.332245,0.148086,-0.284804,-0.276317,-0.014198
quality_min,0.029436,-0.568676,0.785649,1.000000,0.294560,-0.983021,0.615733,0.602014,0.526560,0.774260,...,-0.146110,0.146717,-0.146110,0.014497,-0.041455,-0.101121,0.106119,-0.068429,-0.108281,-0.104164
quality_max,-0.014707,-0.284935,0.628587,0.294560,1.000000,-0.291470,0.206438,0.203001,0.187780,0.211232,...,-0.418021,0.254976,-0.418021,-0.417268,-0.341338,-0.428369,0.026392,-0.414289,-0.379368,-0.015936
quality_std,-0.039298,0.618199,-0.823734,-0.983021,-0.291470,1.000000,-0.611291,-0.597470,-0.523138,-0.773679,...,0.146294,-0.197668,0.146294,-0.016257,0.045359,0.104869,-0.112013,0.069398,0.107982,0.112660
cum_reward_avg,0.008819,-0.416448,0.538676,0.615733,0.206438,-0.611291,1.000000,0.995714,0.945317,0.773693,...,-0.182339,0.131866,-0.182339,-0.047486,-0.070397,-0.124780,-0.262579,-0.066568,-0.100364,0.153011
cum_reward_max,0.004996,-0.402147,0.525389,0.602014,0.203001,-0.597470,0.995714,1.000000,0.961596,0.754861,...,-0.184092,0.127402,-0.184092,-0.052014,-0.073080,-0.124833,-0.283958,-0.067886,-0.101959,0.144923
cum_reward_std,0.007599,-0.336589,0.463892,0.526560,0.187780,-0.523138,0.945317,0.961596,1.000000,0.661502,...,-0.177913,0.116856,-0.177913,-0.057681,-0.070426,-0.113382,-0.330283,-0.067986,-0.102209,0.104824
mom_reward_avg,0.001519,-0.532794,0.687860,0.774260,0.211232,-0.773679,0.773693,0.754861,0.661502,1.000000,...,-0.184039,0.177985,-0.184039,-0.003229,-0.050367,-0.113662,0.093544,-0.072815,-0.109789,0.185404


## Weighted

In [97]:
fi = pd.read_csv('data/generated/fi_2020-2023.csv', index_col=0)
fi.rename(index={'Lower_limit': 'Param 1', 'Upper_limit': 'Param 2'}, inplace=True)
fi = fi / fi.max()
fi = fi.to_dict()['importance']
fi

{'HR Usage Rate': 1.0,
 'TCH Blocking Rate, BH': 0.05306688095919128,
 'Number of Available\nTCH': 0.9145249190288962,
 'TCH Traffic (Erl), BH': 0.48950080102867455,
 'Param 1': 0.934843836886342,
 'Param 2': 0.8525678849265464}

In [98]:
get_weighted_mean = partial(get_weighted_aggregation, fi=fi, method=np.sum)

In [99]:
df_weighted = orig_df[['cell_id']].copy()

df_weighted[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_weighted_mean, x)))
df_weighted

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,3.977416,4.088273e-03,3.977416,20.452360,32.004431,2.794267,0.000054,3.022653,17.056337,0.026398
1,22944,7.044659,4.990282e-04,7.044659,23.046348,33.491390,2.987221,0.000054,3.242961,26.542095,0.012049
2,26335,3.799549,6.917724e-30,3.799549,18.993445,28.726259,2.727743,0.000054,2.943685,16.232136,0.194804
3,26332,4.064677,6.557590e-13,4.064677,19.940895,30.928673,2.684200,0.000054,2.882820,17.064314,0.009713
4,26336,4.288557,3.184883e-02,4.288557,21.115263,31.305614,2.845763,0.000054,3.024424,17.891876,0.018692
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,3.488157,8.134623e-03,3.488157,17.234414,24.606040,2.685131,0.000062,2.688601,14.475203,0.017136
990,5682,3.272016,3.829992e-03,3.272016,19.933170,27.829984,2.782681,0.000062,2.672843,13.133915,0.399837
991,12772,4.296801,1.639180e-23,4.296801,22.625169,33.255036,2.972496,0.000073,3.164523,18.203419,0.018272
992,12771,3.711831,1.976281e-03,3.711831,17.890696,23.978053,2.554521,0.000073,2.487802,15.204722,0.026221


In [100]:
merged = rewards.merge(df_weighted, left_on='cell_id', right_on='cell_id', how='inner')
merged.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,0.017573,0.009130,0.017573,0.023596,-0.004389,0.010561,-0.013959,0.015103,0.016296,-0.038950
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,0.097203,-0.237242,0.097203,0.108176,0.126682,0.152036,-0.042587,0.115123,0.140761,0.043791
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,-0.218367,0.177167,-0.218367,-0.192197,-0.209108,-0.237964,0.023210,-0.187263,-0.224754,-0.040953
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,-0.055198,0.017541,-0.055198,-0.007003,-0.049867,-0.060698,0.050217,-0.027469,-0.074103,-0.038077
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,-0.440871,0.161293,-0.440871,-0.495184,-0.394346,-0.484783,-0.043007,-0.435226,-0.426809,0.009177
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,0.051539,-0.053727,0.051539,0.002137,0.048255,0.049537,-0.036818,0.018146,0.069140,0.038016
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,-0.071043,0.078705,-0.071043,-0.024641,-0.044870,-0.041092,-0.229756,0.000779,-0.056655,-0.011181
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,-0.069228,0.080030,-0.069228,-0.024867,-0.044238,-0.036326,-0.420374,0.003255,-0.055142,-0.004983
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,-0.058445,0.067664,-0.058445,-0.029643,-0.037538,-0.024048,-0.699903,-0.004574,-0.045130,0.000916
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,-0.072226,0.081552,-0.072226,-0.009314,-0.042213,-0.025731,0.029132,0.001893,-0.058037,-0.027096


## Adaptive threshold

In [101]:
df_thresh = orig_df[['cell_id']].copy()

df_thresh[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_thresh_sum, x)))
df_thresh

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,7.000000,0.166667,7.000000,37.833333,59.333333,5.000000,0.0,5.333333,31.166667,1.500000
1,22944,13.500000,0.000000,13.500000,46.000000,66.666667,5.500000,0.0,6.166667,50.666667,0.666667
2,26335,7.000000,0.000000,7.000000,34.833333,52.833333,4.666667,0.0,5.166667,30.500000,2.666667
3,26332,7.666667,0.000000,7.666667,37.000000,57.500000,4.833333,0.0,5.166667,32.000000,0.333333
4,26336,8.000000,2.000000,8.000000,40.666667,60.000000,5.166667,0.0,5.333333,33.500000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,6.166667,0.500000,6.166667,32.333333,46.000000,4.833333,0.0,4.666667,26.666667,1.000000
990,5682,6.000000,0.166667,6.000000,37.333333,52.333333,5.000000,0.0,4.666667,24.500000,2.166667
991,12772,8.333333,0.000000,8.333333,43.666667,64.333333,5.500000,0.0,5.666667,35.166667,0.833333
992,12771,6.833333,0.000000,6.833333,33.000000,44.666667,4.500000,0.0,4.333333,28.333333,1.500000


In [102]:
merged_thresh = rewards.merge(df_thresh, left_on='cell_id', right_on='cell_id', how='inner')
merged_thresh.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,0.014036,0.005643,0.014036,0.027610,-0.000669,-0.014985,NaN,0.026798,0.012548,-0.004771
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,0.175184,-0.215700,0.175184,0.048352,0.089823,0.151874,NaN,0.091773,0.151945,-0.090297
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,-0.357426,0.160971,-0.357426,-0.158185,-0.196870,-0.287780,NaN,-0.205309,-0.256556,0.185042
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,-0.159107,0.001548,-0.159107,0.030098,-0.026508,-0.103673,NaN,-0.035293,-0.085222,0.132811
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,-0.464991,0.148022,-0.464991,-0.458375,-0.385747,-0.473477,NaN,-0.419358,-0.439653,0.024481
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,0.183933,-0.038463,0.183933,-0.034921,0.026324,0.111122,NaN,0.035314,0.083577,-0.185098
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,-0.218080,0.070949,-0.218080,-0.002802,-0.036455,-0.109175,NaN,-0.026757,-0.072041,0.202109
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,-0.211665,0.073039,-0.211665,-0.010960,-0.041082,-0.113546,NaN,-0.025675,-0.072635,0.181275
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,-0.155647,0.063993,-0.155647,-0.036306,-0.047657,-0.098319,NaN,-0.029484,-0.063955,0.082754
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,-0.231018,0.073735,-0.231018,0.014274,-0.032292,-0.097886,NaN,-0.028652,-0.074152,0.202957


## Adaptive weighted threshold

In [103]:
df_weighted_thresh = orig_df[['cell_id']].copy()

df_weighted_thresh[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_weighted_thresh_sum, x)))
df_weighted_thresh

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,7.000000,0.166667,7.000000,37.833333,59.333333,5.000000,0.0,5.333333,31.166667,1.500000
1,22944,13.500000,0.000000,13.500000,46.000000,66.666667,5.500000,0.0,6.166667,50.666667,0.666667
2,26335,7.000000,0.000000,7.000000,34.833333,52.833333,4.666667,0.0,5.166667,30.500000,2.666667
3,26332,7.666667,0.000000,7.666667,37.000000,57.500000,4.833333,0.0,5.166667,32.000000,0.333333
4,26336,8.000000,2.000000,8.000000,40.666667,60.000000,5.166667,0.0,5.333333,33.500000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,6.166667,0.500000,6.166667,32.333333,46.000000,4.833333,0.0,4.666667,26.666667,1.000000
990,5682,6.000000,0.166667,6.000000,37.333333,52.333333,5.000000,0.0,4.666667,24.500000,2.166667
991,12772,8.333333,0.000000,8.333333,43.666667,64.333333,5.500000,0.0,5.666667,35.166667,0.833333
992,12771,6.833333,0.000000,6.833333,33.000000,44.666667,4.500000,0.0,4.333333,28.333333,1.500000


In [104]:
df_weighted_thresh = rewards.merge(df_weighted_thresh, left_on='cell_id', right_on='cell_id', how='inner')
df_weighted_thresh.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,0.014036,0.005643,0.014036,0.027610,-0.000669,-0.014985,NaN,0.026798,0.012548,-0.004771
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,0.175184,-0.215700,0.175184,0.048352,0.089823,0.151874,NaN,0.091773,0.151945,-0.090297
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,-0.357426,0.160971,-0.357426,-0.158185,-0.196870,-0.287780,NaN,-0.205309,-0.256556,0.185042
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,-0.159107,0.001548,-0.159107,0.030098,-0.026508,-0.103673,NaN,-0.035293,-0.085222,0.132811
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,-0.464991,0.148022,-0.464991,-0.458375,-0.385747,-0.473477,NaN,-0.419358,-0.439653,0.024481
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,0.183933,-0.038463,0.183933,-0.034921,0.026324,0.111122,NaN,0.035314,0.083577,-0.185098
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,-0.218080,0.070949,-0.218080,-0.002802,-0.036455,-0.109175,NaN,-0.026757,-0.072041,0.202109
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,-0.211665,0.073039,-0.211665,-0.010960,-0.041082,-0.113546,NaN,-0.025675,-0.072635,0.181275
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,-0.155647,0.063993,-0.155647,-0.036306,-0.047657,-0.098319,NaN,-0.029484,-0.063955,0.082754
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,-0.231018,0.073735,-0.231018,0.014274,-0.032292,-0.097886,NaN,-0.028652,-0.074152,0.202957


# Sampled | no ref window


In [105]:
orig_df = pd.read_csv('data/generated/drift/by_cell_agent/run_8/by_train_regressive__sampled-drift-1000_no-window_.csv', index_col=0)
orig_df

,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test,cell_id
0,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",25771
1,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",22944
2,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",26335
3,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",26332
4,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",26336
...,...,...,...,...,...,...,...,...,...,...,...
989,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",5683
990,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ..

In [106]:
stat_tests = [c for c in orig_df.columns if c not in ['cell_id']]

drift_scores_df = orig_df[['cell_id']].copy()

drift_scores_df[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_total_score, x)))
drift_scores_df

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,0.833333,0.833333,1.000000,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333
1,22944,0.833333,0.833333,1.000000,0.833333,0.833333,0.833333,1.000000,0.833333,0.833333,1.000000
2,26335,1.000000,1.000000,1.000000,0.833333,0.833333,0.833333,1.000000,0.833333,1.000000,0.833333
3,26332,1.000000,1.000000,0.833333,0.833333,0.833333,0.833333,1.000000,0.833333,1.000000,0.833333
4,26336,0.833333,0.833333,1.000000,0.833333,0.833333,0.833333,0.833333,0.833333,1.000000,0.833333
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,0.833333,0.833333,1.000000,1.000000,1.000000,1.000000,0.833333,0.833333,0.833333,0.833333
990,5682,0.833333,0.833333,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.833333,0.666667
991,12772,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.833333,1.000000,0.833333
992,12771,0.833333,0.833333,1.000000,1.000000,1.000000,1.000000,0.833333,0.833333,1.000000,0.833333


## Regress

In [107]:
drift_regress_df = orig_df[['cell_id']].copy()

drift_regress_df[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_regress_score, x)))
drift_regress_df

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,3.079131e-01,3.079131e-01,4.722307,24.082304,33.420538,3.240035,5.997323e-01,3.647950,19.233543,0.300340
1,22944,5.118422e-02,5.118422e-02,8.025195,27.685188,37.685185,3.444991,4.313094e-03,3.791377,28.990798,0.017917
2,26335,6.851186e-19,6.851186e-19,4.638261,22.144719,30.847093,3.038247,1.376431e-08,3.435655,19.269641,0.626102
3,26332,1.557643e-05,1.557643e-05,4.843760,24.054648,34.649028,3.128291,1.881595e-05,3.137929,19.579952,0.588086
4,26336,9.408428e-01,9.408428e-01,5.134385,27.714290,40.759644,3.496689,4.612117e-01,3.714086,20.356294,0.069908
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,5.978521e-01,5.978521e-01,4.372890,19.661666,26.520277,3.149742,1.485134e-01,3.354724,17.385946,0.079615
990,5682,4.246977e-01,4.246977e-01,4.022376,21.141692,29.058497,3.275769,8.441780e-02,3.436627,15.521926,0.657484
991,12772,4.561161e-13,4.561161e-13,5.345316,28.000094,41.591981,3.941070,6.541882e-09,3.871215,22.074685,0.478839
992,12771,3.889566e-01,3.889566e-01,4.186046,19.951571,27.358941,3.183168,1.133992e-01,3.019522,15.583456,0.192811


In [108]:
merged = rewards.merge(drift_scores_df, left_on='cell_id', right_on='cell_id', how='inner')
merged.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,-0.001354,-0.001354,0.028919,0.002911,-0.024025,-0.004324,-0.008335,-0.050606,-0.019135,0.019884
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,0.588531,0.588531,0.284234,0.022122,0.189125,0.176113,0.306818,0.353520,0.595053,0.143703
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,-0.424538,-0.424538,-0.190444,-0.039181,-0.199857,-0.191157,-0.203164,-0.377565,-0.414596,-0.164997
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,-0.255248,-0.255248,-0.102831,0.051600,-0.157566,-0.126014,-0.052265,-0.319663,-0.338615,-0.068507
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,-0.279873,-0.279873,-0.087490,-0.031366,-0.083250,-0.081799,-0.159179,-0.165850,-0.234196,-0.135416
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,0.273979,0.273979,0.144991,-0.031526,0.190877,0.168346,0.066317,0.378150,0.373403,0.110883
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,-0.134304,-0.134304,-0.126914,-0.017974,-0.176619,-0.178549,-0.040758,-0.297126,-0.248986,-0.123675
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,-0.113230,-0.113230,-0.119712,-0.036196,-0.185314,-0.187395,-0.033857,-0.287874,-0.234487,-0.117879
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,-0.044786,-0.044786,-0.067688,-0.067855,-0.152582,-0.159116,-0.022443,-0.191684,-0.139756,-0.063917
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,-0.151409,-0.151409,-0.129724,0.004750,-0.163469,-0.161461,-0.032783,-0.303016,-0.244599,-0.132228


In [109]:
merged = rewards.merge(drift_regress_df, left_on='cell_id', right_on='cell_id', how='inner')
merged.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,0.010639,0.010639,0.007576,-0.009274,-0.021746,-0.018501,-0.011451,-0.014602,0.008715,-0.024804
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,-0.402653,-0.402653,0.335901,0.107460,0.165867,0.264009,-0.225687,0.155137,0.157357,-0.013888
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,0.297615,0.297615,-0.589596,-0.210046,-0.266939,-0.351282,0.141016,-0.273001,-0.256067,0.056579
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,0.120718,0.120718,-0.380062,-0.009968,-0.060353,-0.115577,-0.004668,-0.037538,-0.091151,-0.052659
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,0.218051,0.218051,-0.409225,-0.529890,-0.517721,-0.508303,0.113757,-0.580612,-0.435715,0.036375
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,-0.152431,-0.152431,0.453150,-0.004484,0.050613,0.110168,-0.010642,0.035832,0.089215,-0.000251
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,0.113750,0.113750,-0.429257,-0.002089,-0.038938,-0.077639,0.021199,-0.037631,-0.071612,0.061397
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,0.111734,0.111734,-0.421586,-0.015745,-0.051952,-0.089582,0.030078,-0.045912,-0.071239,0.053611
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,0.087194,0.087194,-0.305814,-0.051637,-0.076613,-0.099376,0.051823,-0.068963,-0.060995,0.020542
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,0.114607,0.114607,-0.487808,0.013900,-0.026984,-0.057711,0.004255,-0.022134,-0.075014,0.070669


In [110]:
merged.corr(method='kendall')

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.038715,0.008395,0.019957,-0.010198,-0.026838,0.006847,0.003968,0.005609,0.001734,...,0.014291,0.014291,0.014705,0.008982,0.003811,0.009987,0.002863,0.005343,0.014295,-0.011426
drift_score,-0.038715,1.000000,-0.563385,-0.464556,-0.232765,0.505011,-0.378430,-0.367684,-0.307606,-0.499002,...,-0.436725,-0.436725,0.242535,0.092677,0.137358,0.193268,-0.386900,0.133570,0.122887,-0.072649
quality_avg,0.008395,-0.563385,1.000000,0.590028,0.454029,-0.632093,0.419421,0.412733,0.357276,0.558258,...,0.273840,0.273840,-0.297157,-0.187901,-0.224657,-0.255878,0.204455,-0.222574,-0.181680,-0.033832
quality_min,0.019957,-0.464556,0.590028,1.000000,0.196885,-0.884325,0.487317,0.477523,0.396062,0.637737,...,0.121610,0.121610,-0.184185,-0.030303,-0.069855,-0.100362,0.070832,-0.050857,-0.079650,-0.129200
quality_max,-0.010198,-0.232765,0.454029,0.196885,1.000000,-0.193189,0.154320,0.152633,0.140335,0.162721,...,0.176551,0.176551,-0.275994,-0.327502,-0.317889,-0.308495,0.130392,-0.371269,-0.243230,0.028471
quality_std,-0.026838,0.505011,-0.632093,-0.884325,-0.193189,1.000000,-0.490762,-0.482335,-0.397162,-0.648789,...,-0.151766,-0.151766,0.188610,0.030400,0.073405,0.105817,-0.093558,0.050808,0.079755,0.127360
cum_reward_avg,0.006847,-0.378430,0.419421,0.487317,0.154320,-0.490762,1.000000,0.963841,0.834348,0.731017,...,0.113295,0.113295,-0.199545,-0.058977,-0.082897,-0.103541,0.091850,-0.073061,-0.077319,0.007944
cum_reward_max,0.003968,-0.367684,0.412733,0.477523,0.152633,-0.482335,0.963841,1.000000,0.877907,0.728345,...,0.110150,0.110150,-0.197882,-0.061501,-0.084249,-0.104761,0.087798,-0.074061,-0.079145,0.005789
cum_reward_std,0.005609,-0.307606,0.357276,0.396062,0.140335,-0.397162,0.834348,0.877907,1.000000,0.587692,...,0.101535,0.101535,-0.174431,-0.067928,-0.084504,-0.101291,0.078713,-0.074318,-0.078822,-0.007226
mom_reward_avg,0.001734,-0.499002,0.558258,0.637737,0.162721,-0.648789,0.731017,0.728345,0.587692,1.000000,...,0.135238,0.135238,-0.233973,-0.026533,-0.061537,-0.089096,0.107102,-0.063611,-0.085264,0.011011


In [111]:
merged.corr(method='spearman')

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.047393,0.013536,0.029436,-0.014707,-0.039298,0.008819,0.004996,0.007599,0.001519,...,0.021158,0.021158,0.021985,0.013284,0.006287,0.014013,0.004972,0.007882,0.022096,-0.016269
drift_score,-0.047393,1.000000,-0.689656,-0.568676,-0.284935,0.618199,-0.416448,-0.402147,-0.336589,-0.532794,...,-0.534010,-0.534010,0.296894,0.113449,0.168144,0.236585,-0.473616,0.163506,0.150429,-0.088932
quality_avg,0.013536,-0.689656,1.000000,0.785649,0.628587,-0.823734,0.538676,0.525389,0.463892,0.687860,...,0.410713,0.410713,-0.441750,-0.284429,-0.335490,-0.375792,0.316890,-0.342010,-0.282388,-0.038906
quality_min,0.029436,-0.568676,0.785649,1.000000,0.294560,-0.983021,0.615733,0.602014,0.526560,0.774260,...,0.190110,0.190110,-0.273402,-0.046382,-0.105522,-0.149445,0.115255,-0.077130,-0.120462,-0.173799
quality_max,-0.014707,-0.284935,0.628587,0.294560,1.000000,-0.291470,0.206438,0.203001,0.187780,0.211232,...,0.261356,0.261356,-0.422805,-0.472391,-0.457898,-0.446872,0.196947,-0.534808,-0.374099,0.043577
quality_std,-0.039298,0.618199,-0.823734,-0.983021,-0.291470,1.000000,-0.611291,-0.597470,-0.523138,-0.773679,...,-0.236429,-0.236429,0.279101,0.046161,0.109935,0.157597,-0.151867,0.077976,0.120525,0.167438
cum_reward_avg,0.008819,-0.416448,0.538676,0.615733,0.206438,-0.611291,1.000000,0.995714,0.945317,0.773693,...,0.152651,0.152651,-0.266344,-0.080133,-0.112550,-0.139261,0.123983,-0.098350,-0.104972,0.012883
cum_reward_max,0.004996,-0.402147,0.525389,0.602014,0.203001,-0.597470,0.995714,1.000000,0.961596,0.754861,...,0.148062,0.148062,-0.262781,-0.083831,-0.114186,-0.140844,0.118220,-0.098987,-0.106768,0.009579
cum_reward_std,0.007599,-0.336589,0.463892,0.526560,0.187780,-0.523138,0.945317,0.961596,1.000000,0.661502,...,0.138362,0.138362,-0.233986,-0.091818,-0.113866,-0.136484,0.106840,-0.099007,-0.105600,-0.010218
mom_reward_avg,0.001519,-0.532794,0.687860,0.774260,0.211232,-0.773679,0.773693,0.754861,0.661502,1.000000,...,0.177786,0.177786,-0.303065,-0.034730,-0.081096,-0.117266,0.140338,-0.083662,-0.112239,0.018796


## Weighted

In [112]:
fi = pd.read_csv('data/generated/fi_2020-2023.csv', index_col=0)
fi.rename(index={'Lower_limit': 'Param 1', 'Upper_limit': 'Param 2'}, inplace=True)
fi = fi / fi.max()
fi = fi.to_dict()['importance']
fi

{'HR Usage Rate': 1.0,
 'TCH Blocking Rate, BH': 0.05306688095919128,
 'Number of Available\nTCH': 0.9145249190288962,
 'TCH Traffic (Erl), BH': 0.48950080102867455,
 'Param 1': 0.934843836886342,
 'Param 2': 0.8525678849265464}

In [113]:
get_weighted_mean = partial(get_weighted_aggregation, fi=fi, method=np.sum)

In [114]:
df_weighted = orig_df[['cell_id']].copy()

df_weighted[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_weighted_mean, x)))
df_weighted

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,1.633999e-02,1.633999e-02,4.029667,21.366584,29.552923,2.718403,3.182593e-02,3.065037,17.248688,0.016212
1,22944,2.716187e-03,2.716187e-03,6.593849,22.859572,31.166296,2.805924,2.289159e-04,3.083434,25.013443,0.000951
2,26335,8.154165e-20,8.154165e-20,3.910988,19.834605,27.534005,2.592289,1.197789e-08,2.935333,16.899078,0.052183
3,26332,8.266015e-07,8.266015e-07,4.095031,21.032704,30.419708,2.660322,1.015362e-06,2.669777,17.083524,0.032158
4,26336,4.992759e-02,4.992759e-02,4.271981,23.624190,35.096746,2.922867,2.447508e-02,3.073695,17.843151,0.007580
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,3.172614e-02,3.172614e-02,3.635384,16.284754,22.056833,2.491108,7.881150e-03,2.776852,15.421736,0.004227
990,5682,2.253738e-02,2.253738e-02,3.284668,18.007659,24.716205,2.602252,4.479797e-03,2.732143,13.632681,0.554719
991,12772,4.171296e-13,4.171296e-13,4.385296,23.473197,32.708054,2.884853,3.776105e-09,3.215354,18.857023,0.026845
992,12771,2.064072e-02,2.064072e-02,3.438170,17.236805,22.756305,2.427005,6.017887e-03,2.475873,13.466465,0.010275


In [115]:
merged = rewards.merge(df_weighted, left_on='cell_id', right_on='cell_id', how='inner')
merged.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,0.003145,0.003145,0.004691,-0.010761,-0.022527,-0.015619,-0.021660,-0.013997,0.010433,-0.021968
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,-0.379646,-0.379646,0.135997,0.123515,0.124145,0.154509,-0.217631,0.138069,0.147957,0.032329
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,0.278472,0.278472,-0.266748,-0.189598,-0.191926,-0.213463,0.136583,-0.214570,-0.224669,-0.042652
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,0.112469,0.112469,-0.096508,-0.025115,-0.040586,-0.056480,0.001258,-0.030177,-0.081580,-0.053588
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,0.210485,0.210485,-0.448952,-0.523476,-0.480063,-0.489761,0.115349,-0.568401,-0.422510,0.011734
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,-0.140424,-0.140424,0.101305,0.008204,0.025178,0.034297,-0.014450,0.013288,0.075933,0.050872
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,0.100654,0.100654,-0.109548,-0.007912,-0.021556,-0.010470,0.018686,-0.010261,-0.055916,-0.039789
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,0.095485,0.095485,-0.105529,-0.013221,-0.027122,-0.012417,0.021222,-0.012169,-0.052914,-0.030187
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,0.067984,0.067984,-0.081489,-0.031874,-0.042173,-0.020472,0.030562,-0.028387,-0.040359,-0.004637
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,0.106141,0.106141,-0.119219,0.007890,-0.007752,0.006137,0.011461,0.008299,-0.058802,-0.045852


## Adaptive threshold

In [116]:
df_thresh = orig_df[['cell_id']].copy()

df_thresh[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_thresh_sum, x)))
df_thresh

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,1.000000,1.000000,7.500000,39.666667,55.166667,4.666667,0.833333,5.500000,31.666667,1.000000
1,22944,0.166667,0.166667,12.666667,45.500000,62.166667,5.000000,0.000000,5.833333,47.666667,0.000000
2,26335,0.000000,0.000000,7.333333,36.500000,51.000000,4.500000,0.000000,5.166667,31.666667,2.000000
3,26332,0.000000,0.000000,7.833333,39.666667,57.166667,4.833333,0.000000,4.666667,32.333333,1.833333
4,26336,3.000000,3.000000,8.166667,45.666667,67.333333,5.333333,0.666667,5.500000,33.500000,0.166667
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,1.833333,1.833333,6.666667,32.333333,43.833333,4.666667,0.166667,5.000000,28.333333,0.166667
990,5682,1.333333,1.333333,6.166667,34.666667,48.000000,5.000000,0.000000,5.000000,25.333333,2.000000
991,12772,0.000000,0.000000,8.333333,46.000000,68.833333,5.833333,0.000000,6.000000,36.500000,1.500000
992,12771,1.166667,1.166667,6.500000,32.833333,45.166667,5.000000,0.166667,4.500000,25.666667,0.500000


In [117]:
merged_thresh = rewards.merge(df_thresh, left_on='cell_id', right_on='cell_id', how='inner')
merged_thresh.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,0.012006,0.012006,0.007717,-0.009285,-0.022121,-0.011893,-0.010270,-0.016393,0.008934,-0.023762
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,-0.397060,-0.397060,0.333898,0.106558,0.165478,0.266755,-0.199203,0.169164,0.158682,-0.010711
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,0.293490,0.293490,-0.588523,-0.209760,-0.267157,-0.363177,0.119967,-0.285686,-0.256315,0.050682
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,0.118232,0.118232,-0.377821,-0.009785,-0.060268,-0.132654,-0.018863,-0.054121,-0.092284,-0.055041
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,0.214027,0.214027,-0.409720,-0.530101,-0.517892,-0.518138,0.100457,-0.572099,-0.434890,0.029091
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,-0.150232,-0.150232,0.450706,-0.004815,0.050455,0.128499,0.005200,0.053163,0.090172,0.002443
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,0.113538,0.113538,-0.427551,-0.002278,-0.038644,-0.099169,0.012202,-0.049380,-0.071984,0.059270
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,0.111846,0.111846,-0.419613,-0.015782,-0.051915,-0.110932,0.021806,-0.059747,-0.071709,0.050915
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,0.088076,0.088076,-0.304137,-0.051261,-0.077131,-0.114161,0.046524,-0.081009,-0.061508,0.017378
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,0.114543,0.114543,-0.485821,0.013538,-0.026719,-0.077027,-0.005658,-0.034334,-0.075120,0.068321


## Adaptive weighted threshold

In [118]:
df_weighted_thresh = orig_df[['cell_id']].copy()

df_weighted_thresh[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_weighted_thresh_sum, x)))
df_weighted_thresh

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,1.000000,1.000000,7.500000,39.666667,55.166667,4.666667,0.833333,5.500000,31.666667,1.000000
1,22944,0.166667,0.166667,12.666667,45.500000,62.166667,5.000000,0.000000,5.833333,47.666667,0.000000
2,26335,0.000000,0.000000,7.333333,36.500000,51.000000,4.500000,0.000000,5.166667,31.666667,2.000000
3,26332,0.000000,0.000000,7.833333,39.666667,57.166667,4.833333,0.000000,4.666667,32.333333,1.833333
4,26336,3.000000,3.000000,8.166667,45.666667,67.333333,5.333333,0.666667,5.500000,33.500000,0.166667
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,1.833333,1.833333,6.666667,32.333333,43.833333,4.666667,0.166667,5.000000,28.333333,0.166667
990,5682,1.333333,1.333333,6.166667,34.666667,48.000000,5.000000,0.000000,5.000000,25.333333,2.000000
991,12772,0.000000,0.000000,8.333333,46.000000,68.833333,5.833333,0.000000,6.000000,36.500000,1.500000
992,12771,1.166667,1.166667,6.500000,32.833333,45.166667,5.000000,0.166667,4.500000,25.666667,0.500000


In [119]:
df_weighted_thresh = rewards.merge(df_weighted_thresh, left_on='cell_id', right_on='cell_id', how='inner')
df_weighted_thresh.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,0.012006,0.012006,0.007717,-0.009285,-0.022121,-0.011893,-0.010270,-0.016393,0.008934,-0.023762
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,-0.397060,-0.397060,0.333898,0.106558,0.165478,0.266755,-0.199203,0.169164,0.158682,-0.010711
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,0.293490,0.293490,-0.588523,-0.209760,-0.267157,-0.363177,0.119967,-0.285686,-0.256315,0.050682
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,0.118232,0.118232,-0.377821,-0.009785,-0.060268,-0.132654,-0.018863,-0.054121,-0.092284,-0.055041
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,0.214027,0.214027,-0.409720,-0.530101,-0.517892,-0.518138,0.100457,-0.572099,-0.434890,0.029091
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,-0.150232,-0.150232,0.450706,-0.004815,0.050455,0.128499,0.005200,0.053163,0.090172,0.002443
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,0.113538,0.113538,-0.427551,-0.002278,-0.038644,-0.099169,0.012202,-0.049380,-0.071984,0.059270
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,0.111846,0.111846,-0.419613,-0.015782,-0.051915,-0.110932,0.021806,-0.059747,-0.071709,0.050915
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,0.088076,0.088076,-0.304137,-0.051261,-0.077131,-0.114161,0.046524,-0.081009,-0.061508,0.017378
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,0.114543,0.114543,-0.485821,0.013538,-0.026719,-0.077027,-0.005658,-0.034334,-0.075120,0.068321


# NOT Sampled | ref window 1000


In [120]:
orig_df = pd.read_csv('data/generated/drift/by_cell_agent/run_9/by_train_regressive_sampled_ref_sampled_drift_None_window_1k_.csv', index_col=0)
orig_df

,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test,cell_id
0,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",25771
1,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",22944
2,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",26335
3,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",26332
4,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",26336
...,...,...,...,...,...,...,...,...,...,...,...
989,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",5683
990,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ..

In [121]:
stat_tests = [c for c in orig_df.columns if c not in ['cell_id']]

drift_scores_df = orig_df[['cell_id']].copy()

drift_scores_df[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_total_score, x)))
drift_scores_df

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,0.833333,0.833333,0.833333,0.333333,0.333333,0.666667,0.833333,0.666667,0.833333,0.333333
1,22944,0.833333,0.833333,0.833333,0.666667,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333
2,26335,1.000000,1.000000,0.833333,0.333333,0.666667,0.666667,1.000000,0.666667,0.833333,0.500000
3,26332,0.833333,0.833333,0.833333,0.333333,0.333333,0.500000,1.000000,0.500000,0.833333,0.333333
4,26336,0.833333,0.833333,0.833333,0.333333,0.500000,0.666667,0.833333,0.666667,0.833333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,0.833333,0.833333,0.833333,0.000000,0.166667,0.333333,0.833333,0.166667,0.833333,0.500000
990,5682,0.833333,0.833333,0.833333,0.000000,0.166667,0.333333,0.833333,0.166667,0.833333,0.333333
991,12772,0.833333,0.833333,0.833333,0.000000,0.333333,0.333333,1.000000,0.333333,0.833333,0.333333
992,12771,0.666667,0.666667,0.500000,0.000000,0.000000,0.000000,0.833333,0.000000,0.833333,0.500000


In [122]:
merged = rewards.merge(drift_scores_df, left_on='cell_id', right_on='cell_id', how='inner')
merged.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,0.015601,0.015601,0.041070,0.025474,-0.010786,0.006068,0.001766,-0.013796,-0.039201,0.032943
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,0.475339,0.475339,0.025424,0.046467,0.121326,0.127785,0.558046,0.090650,0.411185,-0.059208
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,-0.363035,-0.363035,-0.167696,-0.139181,-0.227776,-0.210051,-0.401887,-0.167724,-0.658532,-0.067503
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,-0.172905,-0.172905,-0.026208,-0.047355,-0.078718,-0.062490,-0.224097,-0.048254,-0.318144,-0.023860
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,-0.288841,-0.288841,-0.155968,-0.221176,-0.400287,-0.387029,-0.276130,-0.339965,-0.256183,-0.098500
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,0.182195,0.182195,0.057885,0.048518,0.071516,0.045824,0.228868,0.015388,0.404745,0.044666
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,-0.039938,-0.039938,-0.084559,-0.013991,-0.002224,0.030284,-0.079679,0.085593,-0.319357,-0.074955
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,0.002123,0.002123,-0.046364,0.001700,0.042286,0.093565,-0.045885,0.164037,-0.323640,-0.066569
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,0.076139,0.076139,0.044128,0.029085,0.117629,0.189696,0.030012,0.271259,-0.263117,-0.034213
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,-0.083481,-0.083481,-0.126632,-0.045192,-0.042301,-0.016681,-0.095366,0.027903,-0.374736,-0.083465


## Regress

In [123]:
drift_regress_df = orig_df[['cell_id']].copy()

drift_regress_df[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_regress_score, x)))
drift_regress_df

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,0.982398,0.982398,0.960471,0.451576,1.207548,0.735160,0.964230,0.651383,4.016921,1.416430
1,22944,0.993703,0.993703,1.840785,0.564467,1.428007,0.890411,0.649128,0.863567,6.645657,0.928893
2,26335,0.002921,0.002921,1.030921,0.469831,1.236459,0.786129,0.001701,0.665784,4.065196,1.085661
3,26332,0.129271,0.129271,1.025962,0.439040,1.213825,0.731236,0.041973,0.611452,4.083087,1.338510
4,26336,0.999996,0.999996,1.092780,0.421420,1.013801,0.735792,1.000000,0.732338,4.397999,1.009215
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,1.030972,1.030972,0.660164,0.177187,0.383696,0.467604,0.963062,0.450402,2.648350,1.547301
990,5682,1.008528,1.008528,0.634895,0.211529,0.460843,0.530260,0.857598,0.499305,2.412234,2.479824
991,12772,0.165679,0.165679,0.639066,0.224386,0.567605,0.533942,0.029817,0.497937,2.487203,1.923277
992,12771,1.101117,1.101117,0.571814,0.097409,0.200283,0.368162,0.724677,0.344425,2.160948,1.311986


In [124]:
merged = rewards.merge(drift_regress_df, left_on='cell_id', right_on='cell_id', how='inner')
merged.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,0.002044,0.002044,0.021090,-0.001262,-0.006956,-0.000635,0.010733,-0.000944,0.020219,-0.021600
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,-0.655891,-0.655891,0.144348,0.102174,0.100619,0.145941,-0.528474,0.170504,0.151533,-0.049791
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,0.475799,0.475799,-0.317088,-0.195896,-0.193888,-0.252069,0.372561,-0.321274,-0.256465,0.170458
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,0.281550,0.281550,-0.130931,-0.078290,-0.076862,-0.134011,0.173911,-0.158813,-0.089466,0.149854
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,0.322711,0.322711,-0.435875,-0.277573,-0.271719,-0.302254,0.263727,-0.509997,-0.422890,0.075982
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,-0.296754,-0.296754,0.146476,0.067245,0.065984,0.126664,-0.195504,0.156091,0.086583,-0.188803
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,0.125468,0.125468,-0.130947,0.012455,0.001649,-0.034917,0.098385,-0.076119,-0.023618,0.194885
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,0.092153,0.092153,-0.084261,0.057040,0.037140,0.015838,0.068785,-0.030378,0.018833,0.174953
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,0.009733,0.009733,0.020613,0.125892,0.091248,0.110069,0.005212,0.056018,0.091090,0.081167
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,0.151651,0.151651,-0.195205,-0.047640,-0.050832,-0.096609,0.110890,-0.139095,-0.081895,0.204265


In [125]:
merged.corr(method='kendall')

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.038715,0.008395,0.019957,-0.010198,-0.026838,0.006847,0.003968,0.005609,0.001734,...,0.025752,0.025752,0.012326,0.012569,0.009335,0.012156,0.021282,0.010514,0.010060,0.006628
drift_score,-0.038715,1.000000,-0.563385,-0.464556,-0.232765,0.505011,-0.378430,-0.367684,-0.307606,-0.499002,...,-0.446972,-0.446972,0.127897,0.095108,0.092015,0.130534,-0.393396,0.149391,0.120365,-0.050601
quality_avg,0.008395,-0.563385,1.000000,0.590028,0.454029,-0.632093,0.419421,0.412733,0.357276,0.558258,...,0.307592,0.307592,-0.203969,-0.180935,-0.175310,-0.212042,0.213286,-0.261170,-0.178232,0.059306
quality_min,0.019957,-0.464556,0.590028,1.000000,0.196885,-0.884325,0.487317,0.477523,0.396062,0.637737,...,0.149305,0.149305,-0.087018,-0.068254,-0.068943,-0.094960,0.054747,-0.113087,-0.071962,0.057228
quality_max,-0.010198,-0.232765,0.454029,0.196885,1.000000,-0.193189,0.154320,0.152633,0.140335,0.162721,...,0.206425,0.206425,-0.255938,-0.153669,-0.144770,-0.168080,0.155991,-0.315239,-0.238582,0.072769
quality_std,-0.026838,0.505011,-0.632093,-0.884325,-0.193189,1.000000,-0.490762,-0.482335,-0.397162,-0.648789,...,-0.183095,-0.183095,0.086855,0.076497,0.077121,0.103673,-0.081267,0.112844,0.071971,-0.055169
cum_reward_avg,0.006847,-0.378430,0.419421,0.487317,0.154320,-0.490762,1.000000,0.963841,0.834348,0.731017,...,0.065241,0.065241,-0.034446,0.019681,0.007898,-0.015784,0.058631,-0.033125,0.001593,0.089345
cum_reward_max,0.003968,-0.367684,0.412733,0.477523,0.152633,-0.482335,0.963841,1.000000,0.877907,0.728345,...,0.060780,0.060780,-0.031155,0.025943,0.013409,-0.009195,0.053897,-0.027166,0.004955,0.088534
cum_reward_std,0.005609,-0.307606,0.357276,0.396062,0.140335,-0.397162,0.834348,0.877907,1.000000,0.587692,...,0.047970,0.047970,-0.017639,0.044602,0.030764,0.015041,0.046022,-0.007577,0.015015,0.073827
mom_reward_avg,0.001734,-0.499002,0.558258,0.637737,0.162721,-0.648789,0.731017,0.728345,0.587692,1.000000,...,0.148213,0.148213,-0.126819,-0.075584,-0.078131,-0.111493,0.100049,-0.143920,-0.088601,0.093884


In [126]:
merged.corr(method='spearman')

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.047393,0.013536,0.029436,-0.014707,-0.039298,0.008819,0.004996,0.007599,0.001519,...,0.039372,0.039372,0.016820,0.018687,0.013879,0.018508,0.032231,0.015042,0.014654,0.010336
drift_score,-0.047393,1.000000,-0.689656,-0.568676,-0.284935,0.618199,-0.416448,-0.402147,-0.336589,-0.532794,...,-0.547151,-0.547151,0.156562,0.116425,0.112638,0.159790,-0.481568,0.182874,0.147343,-0.061942
quality_avg,0.013536,-0.689656,1.000000,0.785649,0.628587,-0.823734,0.538676,0.525389,0.463892,0.687860,...,0.454545,0.454545,-0.313200,-0.266581,-0.258160,-0.313027,0.327054,-0.400810,-0.277570,0.087550
quality_min,0.029436,-0.568676,0.785649,1.000000,0.294560,-0.983021,0.615733,0.602014,0.526560,0.774260,...,0.227158,0.227158,-0.130537,-0.103464,-0.104137,-0.142117,0.096797,-0.173012,-0.108977,0.086253
quality_max,-0.014707,-0.284935,0.628587,0.294560,1.000000,-0.291470,0.206438,0.203001,0.187780,0.211232,...,0.314429,0.314429,-0.395554,-0.223887,-0.210567,-0.247222,0.235795,-0.469802,-0.370153,0.114935
quality_std,-0.039298,0.618199,-0.823734,-0.983021,-0.291470,1.000000,-0.611291,-0.597470,-0.523138,-0.773679,...,-0.276935,-0.276935,0.130112,0.116869,0.117317,0.156280,-0.137140,0.172018,0.109195,-0.083149
cum_reward_avg,0.008819,-0.416448,0.538676,0.615733,0.206438,-0.611291,1.000000,0.995714,0.945317,0.773693,...,0.088617,0.088617,-0.046514,0.025773,0.010286,-0.021202,0.080069,-0.046304,0.001691,0.119265
cum_reward_max,0.004996,-0.402147,0.525389,0.602014,0.203001,-0.597470,0.995714,1.000000,0.961596,0.754861,...,0.081220,0.081220,-0.041610,0.034426,0.018101,-0.011771,0.072661,-0.037035,0.006704,0.117724
cum_reward_std,0.007599,-0.336589,0.463892,0.526560,0.187780,-0.523138,0.945317,0.961596,1.000000,0.661502,...,0.062547,0.062547,-0.023539,0.060488,0.042182,0.020690,0.061736,-0.007770,0.020283,0.098640
mom_reward_avg,0.001519,-0.532794,0.687860,0.774260,0.211232,-0.773679,0.773693,0.754861,0.661502,1.000000,...,0.194140,0.194140,-0.165766,-0.099459,-0.102601,-0.146120,0.131642,-0.189318,-0.116377,0.121931


## Weighted

In [127]:
fi = pd.read_csv('data/generated/fi_2020-2023.csv', index_col=0)
fi.rename(index={'Lower_limit': 'Param 1', 'Upper_limit': 'Param 2'}, inplace=True)
fi = fi / fi.max()
fi = fi.to_dict()['importance']
fi

{'HR Usage Rate': 1.0,
 'TCH Blocking Rate, BH': 0.05306688095919128,
 'Number of Available\nTCH': 0.9145249190288962,
 'TCH Traffic (Erl), BH': 0.48950080102867455,
 'Param 1': 0.934843836886342,
 'Param 2': 0.8525678849265464}

In [128]:
get_weighted_mean = partial(get_weighted_aggregation, fi=fi, method=np.sum)

In [129]:
df_weighted = orig_df[['cell_id']].copy()

df_weighted[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_weighted_mean, x)))
df_weighted

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,0.053242,0.053242,0.837810,0.406493,1.090307,0.643900,0.051703,0.579648,3.612372,0.405363
1,22944,0.052733,0.052733,1.540434,0.469304,1.193881,0.734600,0.034447,0.719573,5.758513,0.049293
2,26335,0.001397,0.001397,0.853264,0.414276,1.092814,0.663215,0.000833,0.559662,3.569114,0.680015
3,26332,0.034133,0.034133,0.860315,0.392744,1.091433,0.627425,0.017868,0.522639,3.564637,0.750837
4,26336,0.053067,0.053067,0.924606,0.370442,0.893458,0.628192,0.053067,0.630391,3.865623,0.159107
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,0.080664,0.080664,0.560986,0.160405,0.349551,0.404227,0.053368,0.389940,2.344384,0.574251
990,5682,0.060941,0.060941,0.531448,0.183708,0.404231,0.438665,0.047162,0.418950,2.110993,1.349273
991,12772,0.141562,0.141562,0.520843,0.192651,0.492185,0.447597,0.026094,0.411649,2.121407,1.106324
992,12771,0.144790,0.144790,0.478254,0.079410,0.163883,0.290477,0.086498,0.281411,1.886415,0.376215


In [130]:
merged = rewards.merge(df_weighted, left_on='cell_id', right_on='cell_id', how='inner')
merged.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,-0.052086,-0.052086,0.023455,0.000415,-0.006389,0.002788,-0.043451,-0.001099,0.023530,-0.032025
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,-0.188246,-0.188246,0.104828,0.105228,0.100416,0.116544,-0.191740,0.131213,0.141938,0.068609
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,0.116220,0.116220,-0.221922,-0.184348,-0.181266,-0.196638,0.130541,-0.249846,-0.225963,-0.019741
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,0.016402,0.016402,-0.064799,-0.076243,-0.071693,-0.080497,0.030966,-0.079161,-0.079829,-0.029016
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,0.153119,0.153119,-0.421709,-0.273153,-0.266342,-0.297268,0.142143,-0.515172,-0.410315,0.018571
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,-0.008698,-0.008698,0.058726,0.062880,0.058198,0.065709,-0.032668,0.066975,0.073665,0.017174
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,-0.075390,-0.075390,-0.023423,0.021120,0.012847,0.022843,-0.015538,0.005465,-0.010265,0.001988
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,-0.119091,-0.119091,0.021394,0.065781,0.048043,0.070835,-0.052695,0.045644,0.033461,-0.001386
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,-0.177528,-0.177528,0.096538,0.133957,0.099887,0.145122,-0.115157,0.098843,0.106294,-0.011912
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,-0.039536,-0.039536,-0.079834,-0.038591,-0.038907,-0.036497,0.015322,-0.054514,-0.066426,0.003034


## Adaptive threshold

In [131]:
df_thresh = orig_df[['cell_id']].copy()

df_thresh[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_thresh_sum, x)))
df_thresh

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,3.166667,3.166667,1.166667,0.333333,1.500000,1.000000,1.500000,0.833333,6.166667,4.333333
1,22944,3.166667,3.166667,2.500000,0.666667,2.000000,1.000000,1.000000,1.000000,10.500000,3.000000
2,26335,0.000000,0.000000,1.166667,0.333333,1.833333,1.000000,0.000000,0.833333,6.166667,3.166667
3,26332,0.166667,0.166667,1.166667,0.333333,1.500000,0.833333,0.000000,0.666667,6.333333,4.166667
4,26336,3.166667,3.166667,1.500000,0.333333,1.333333,0.833333,1.500000,1.000000,6.833333,3.166667
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,3.166667,3.166667,0.833333,0.000000,0.166667,0.333333,1.500000,0.166667,4.000000,4.833333
990,5682,3.166667,3.166667,0.833333,0.000000,0.166667,0.333333,1.333333,0.166667,3.500000,7.833333
991,12772,0.333333,0.333333,0.833333,0.000000,0.500000,0.333333,0.000000,0.333333,3.666667,6.000000
992,12771,3.333333,3.333333,0.500000,0.000000,0.000000,0.000000,1.000000,0.000000,3.166667,4.000000


In [132]:
merged_thresh = rewards.merge(df_thresh, left_on='cell_id', right_on='cell_id', how='inner')
merged_thresh.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,0.000159,0.000159,0.020228,0.024791,-0.009876,-0.009887,0.012469,-0.029824,0.023145,-0.021721
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,-0.656955,-0.656955,0.090999,0.053993,0.092202,0.110712,-0.525408,0.119432,0.126421,-0.053490
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,0.473558,0.473558,-0.268451,-0.150925,-0.187368,-0.176784,0.368773,-0.224563,-0.241336,0.172924
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,0.284462,0.284462,-0.075787,-0.048825,-0.066966,-0.072433,0.177382,-0.072122,-0.079582,0.154426
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,0.315738,0.315738,-0.383537,-0.234667,-0.291747,-0.289404,0.254136,-0.457721,-0.414066,0.072927
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,-0.299153,-0.299153,0.089697,0.048272,0.057092,0.057929,-0.198460,0.050718,0.075440,-0.194341
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,0.127201,0.127201,-0.090904,-0.018189,0.003264,0.022585,0.100769,0.039307,-0.016139,0.199106
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,0.094785,0.094785,-0.059984,-0.001104,0.035001,0.079076,0.071944,0.094410,0.025133,0.179202
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,0.013499,0.013499,0.005497,0.029122,0.081323,0.166782,0.008658,0.164439,0.093427,0.083890
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,0.152239,0.152239,-0.143061,-0.049427,-0.044676,-0.036879,0.111800,-0.021760,-0.073354,0.207599


## Adaptive weighted threshold

In [133]:
df_weighted_thresh = orig_df[['cell_id']].copy()

df_weighted_thresh[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_weighted_thresh_sum, x)))
df_weighted_thresh

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,3.166667,3.166667,1.166667,0.333333,1.500000,1.000000,1.500000,0.833333,6.166667,4.333333
1,22944,3.166667,3.166667,2.500000,0.666667,2.000000,1.000000,1.000000,1.000000,10.500000,3.000000
2,26335,0.000000,0.000000,1.166667,0.333333,1.833333,1.000000,0.000000,0.833333,6.166667,3.166667
3,26332,0.166667,0.166667,1.166667,0.333333,1.500000,0.833333,0.000000,0.666667,6.333333,4.166667
4,26336,3.166667,3.166667,1.500000,0.333333,1.333333,0.833333,1.500000,1.000000,6.833333,3.166667
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,3.166667,3.166667,0.833333,0.000000,0.166667,0.333333,1.500000,0.166667,4.000000,4.833333
990,5682,3.166667,3.166667,0.833333,0.000000,0.166667,0.333333,1.333333,0.166667,3.500000,7.833333
991,12772,0.333333,0.333333,0.833333,0.000000,0.500000,0.333333,0.000000,0.333333,3.666667,6.000000
992,12771,3.333333,3.333333,0.500000,0.000000,0.000000,0.000000,1.000000,0.000000,3.166667,4.000000


In [134]:
df_weighted_thresh = rewards.merge(df_weighted_thresh, left_on='cell_id', right_on='cell_id', how='inner')
df_weighted_thresh.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,0.000159,0.000159,0.020228,0.024791,-0.009876,-0.009887,0.012469,-0.029824,0.023145,-0.021721
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,-0.656955,-0.656955,0.090999,0.053993,0.092202,0.110712,-0.525408,0.119432,0.126421,-0.053490
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,0.473558,0.473558,-0.268451,-0.150925,-0.187368,-0.176784,0.368773,-0.224563,-0.241336,0.172924
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,0.284462,0.284462,-0.075787,-0.048825,-0.066966,-0.072433,0.177382,-0.072122,-0.079582,0.154426
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,0.315738,0.315738,-0.383537,-0.234667,-0.291747,-0.289404,0.254136,-0.457721,-0.414066,0.072927
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,-0.299153,-0.299153,0.089697,0.048272,0.057092,0.057929,-0.198460,0.050718,0.075440,-0.194341
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,0.127201,0.127201,-0.090904,-0.018189,0.003264,0.022585,0.100769,0.039307,-0.016139,0.199106
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,0.094785,0.094785,-0.059984,-0.001104,0.035001,0.079076,0.071944,0.094410,0.025133,0.179202
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,0.013499,0.013499,0.005497,0.029122,0.081323,0.166782,0.008658,0.164439,0.093427,0.083890
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,0.152239,0.152239,-0.143061,-0.049427,-0.044676,-0.036879,0.111800,-0.021760,-0.073354,0.207599


# Sampled | ref window 1000

In [135]:
orig_df = pd.read_csv('data/generated/drift/by_cell_agent/run_10/by_train_regressive_sampled_ref_sampled_drift_1000_window_1k_.csv', index_col=0)
orig_df

,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test,cell_id
0,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",25771
1,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",22944
2,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",26335
3,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",26332
4,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",26336
...,...,...,...,...,...,...,...,...,...,...,...
989,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",5683
990,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ..

In [136]:
stat_tests = [c for c in orig_df.columns if c not in ['cell_id']]

drift_scores_df = orig_df[['cell_id']].copy()

drift_scores_df[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_total_score, x)))
drift_scores_df

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,0.833333,0.833333,0.833333,0.333333,0.666667,0.666667,0.833333,0.666667,0.833333,0.333333
1,22944,0.833333,0.833333,0.833333,0.666667,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333
2,26335,1.000000,1.000000,0.833333,0.333333,0.666667,0.666667,1.000000,0.666667,0.833333,0.333333
3,26332,0.833333,0.833333,0.833333,0.333333,0.500000,0.500000,1.000000,0.500000,0.833333,0.500000
4,26336,0.833333,0.833333,0.833333,0.333333,0.500000,0.500000,0.833333,0.500000,0.833333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,0.833333,0.833333,0.833333,0.000000,0.166667,0.333333,0.833333,0.166667,0.833333,0.500000
990,5682,0.833333,0.833333,0.833333,0.000000,0.166667,0.333333,0.833333,0.166667,0.833333,0.333333
991,12772,0.833333,0.833333,0.666667,0.000000,0.333333,0.333333,1.000000,0.333333,0.833333,0.333333
992,12771,0.500000,0.500000,0.500000,0.000000,0.000000,0.000000,0.833333,0.000000,0.833333,0.500000


In [137]:
merged = rewards.merge(drift_scores_df, left_on='cell_id', right_on='cell_id', how='inner')
merged.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,0.020426,0.020426,0.050777,0.012014,0.015075,0.004204,0.015743,0.020163,-0.032771,0.004157
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,0.483210,0.483210,0.043758,0.054121,0.091014,0.158896,0.579473,0.086771,0.417871,-0.061561
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,-0.376859,-0.376859,-0.169591,-0.144724,-0.204794,-0.218141,-0.431804,-0.164857,-0.644308,-0.058966
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,-0.169135,-0.169135,-0.030597,-0.058929,-0.086779,-0.096880,-0.242163,-0.047050,-0.307701,-0.020529
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,-0.292821,-0.292821,-0.169905,-0.214416,-0.391022,-0.399785,-0.277390,-0.334296,-0.243065,-0.083813
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,0.186516,0.186516,0.053594,0.058657,0.073291,0.071260,0.259379,0.016284,0.388794,0.039709
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,-0.039843,-0.039843,-0.062911,-0.021165,-0.014133,0.023681,-0.114763,0.083357,-0.334514,-0.073656
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,0.000968,0.000968,-0.015782,-0.005276,0.031407,0.091598,-0.085073,0.151372,-0.322789,-0.069768
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,0.072364,0.072364,0.075513,0.023326,0.115067,0.201886,-0.012616,0.238750,-0.238053,-0.046022
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,-0.088497,-0.088497,-0.120992,-0.052661,-0.049517,-0.025838,-0.149171,0.027287,-0.375475,-0.084462


## Regress

In [138]:
drift_regress_df = orig_df[['cell_id']].copy()

drift_regress_df[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_regress_score, x)))
drift_regress_df

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,0.956775,0.956775,1.000947,0.459818,1.227711,0.753709,0.869889,0.684726,4.118585,1.010199
1,22944,0.993703,0.993703,1.871576,0.565735,1.432259,0.881881,0.659963,0.867693,6.775952,0.933537
2,26335,0.008560,0.008560,0.972398,0.447312,1.197083,0.744173,0.002929,0.622257,4.027799,1.738669
3,26332,0.095651,0.095651,1.062993,0.450684,1.233759,0.744082,0.031452,0.616991,4.197333,1.067710
4,26336,0.999973,0.999973,1.080400,0.437691,1.056070,0.751227,1.000000,0.733859,4.406156,0.962531
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,1.008668,1.008668,0.712356,0.180921,0.392181,0.480816,0.980576,0.467823,2.736920,0.910720
990,5682,0.996578,0.996578,0.614831,0.204794,0.446266,0.511031,0.604538,0.496222,2.429843,2.640766
991,12772,0.207865,0.207865,0.661447,0.231554,0.584835,0.543354,0.036564,0.504612,2.703010,1.804978
992,12771,1.206066,1.206066,0.508996,0.082378,0.170140,0.322147,1.049649,0.311194,2.015124,1.422181


In [139]:
merged = rewards.merge(drift_regress_df, left_on='cell_id', right_on='cell_id', how='inner')
merged.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,-0.002458,-0.002458,0.028892,-0.002770,-0.008472,0.001189,0.003505,-0.002220,0.023287,-0.056768
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,-0.654083,-0.654083,0.144936,0.105276,0.103171,0.146635,-0.533520,0.175284,0.154800,-0.049773
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,0.471126,0.471126,-0.312807,-0.195999,-0.194386,-0.251328,0.376411,-0.320963,-0.255732,0.165047
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,0.274305,0.274305,-0.124886,-0.079814,-0.078085,-0.129820,0.172852,-0.151544,-0.090040,0.140211
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,0.321323,0.321323,-0.443729,-0.279935,-0.273698,-0.304984,0.258963,-0.511766,-0.434838,0.110777
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,-0.290633,-0.290633,0.140916,0.068604,0.066929,0.124389,-0.195390,0.151355,0.085725,-0.175680
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,0.119125,0.119125,-0.121683,0.011818,0.001602,-0.032761,0.096436,-0.069121,-0.021238,0.185554
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,0.086665,0.086665,-0.076020,0.055559,0.036465,0.017440,0.068441,-0.024708,0.018937,0.160920
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,0.007445,0.007445,0.023873,0.123184,0.089370,0.109717,0.007924,0.056699,0.086640,0.060100
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,0.147492,0.147492,-0.185318,-0.047611,-0.050396,-0.095670,0.115788,-0.133507,-0.076902,0.198603


In [140]:
merged.corr(method='kendall')

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.038715,0.008395,0.019957,-0.010198,-0.026838,0.006847,0.003968,0.005609,0.001734,...,0.025727,0.025727,0.022331,0.011641,0.008674,0.012921,0.006725,0.010401,0.013975,-0.010980
drift_score,-0.038715,1.000000,-0.563385,-0.464556,-0.232765,0.505011,-0.378430,-0.367684,-0.307606,-0.499002,...,-0.452400,-0.452400,0.126045,0.097070,0.094266,0.129569,-0.399886,0.151475,0.123028,-0.065844
quality_avg,0.008395,-0.563385,1.000000,0.590028,0.454029,-0.632093,0.419421,0.412733,0.357276,0.558258,...,0.296715,0.296715,-0.199532,-0.180290,-0.174925,-0.208958,0.218392,-0.261365,-0.178807,0.070504
quality_min,0.019957,-0.464556,0.590028,1.000000,0.196885,-0.884325,0.487317,0.477523,0.396062,0.637737,...,0.139238,0.139238,-0.075237,-0.066629,-0.067513,-0.088481,0.060397,-0.103037,-0.067302,0.044353
quality_max,-0.010198,-0.232765,0.454029,0.196885,1.000000,-0.193189,0.154320,0.152633,0.140335,0.162721,...,0.201837,0.201837,-0.260684,-0.153941,-0.144636,-0.169429,0.151120,-0.322452,-0.242196,0.095321
quality_std,-0.026838,0.505011,-0.632093,-0.884325,-0.193189,1.000000,-0.490762,-0.482335,-0.397162,-0.648789,...,-0.171504,-0.171504,0.075253,0.074078,0.075115,0.096213,-0.084712,0.102907,0.067594,-0.040973
cum_reward_avg,0.006847,-0.378430,0.419421,0.487317,0.154320,-0.490762,1.000000,0.963841,0.834348,0.731017,...,0.067370,0.067370,-0.032906,0.016998,0.006211,-0.013203,0.059845,-0.029208,-0.001634,0.097937
cum_reward_max,0.003968,-0.367684,0.412733,0.477523,0.152633,-0.482335,0.963841,1.000000,0.877907,0.728345,...,0.062335,0.062335,-0.029830,0.022870,0.011399,-0.007001,0.054914,-0.024409,0.001754,0.095863
cum_reward_std,0.005609,-0.307606,0.357276,0.396062,0.140335,-0.397162,0.834348,0.877907,1.000000,0.587692,...,0.049447,0.049447,-0.017572,0.040077,0.026756,0.014783,0.041833,-0.008585,0.010553,0.075573
mom_reward_avg,0.001734,-0.499002,0.558258,0.637737,0.162721,-0.648789,0.731017,0.728345,0.587692,1.000000,...,0.142624,0.142624,-0.121708,-0.076962,-0.078869,-0.107476,0.108329,-0.138401,-0.086325,0.123304


In [141]:
merged.corr(method='spearman')

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.047393,0.013536,0.029436,-0.014707,-0.039298,0.008819,0.004996,0.007599,0.001519,...,0.038601,0.038601,0.032488,0.016638,0.012415,0.018993,0.010301,0.016047,0.020168,-0.015633
drift_score,-0.047393,1.000000,-0.689656,-0.568676,-0.284935,0.618199,-0.416448,-0.402147,-0.336589,-0.532794,...,-0.553796,-0.553796,0.154295,0.118826,0.115393,0.158609,-0.489512,0.185425,0.150602,-0.080602
quality_avg,0.013536,-0.689656,1.000000,0.785649,0.628587,-0.823734,0.538676,0.525389,0.463892,0.687860,...,0.442206,0.442206,-0.308746,-0.266359,-0.258150,-0.309366,0.333467,-0.399266,-0.279376,0.102846
quality_min,0.029436,-0.568676,0.785649,1.000000,0.294560,-0.983021,0.615733,0.602014,0.526560,0.774260,...,0.213028,0.213028,-0.113352,-0.100923,-0.101402,-0.132812,0.101972,-0.156933,-0.101539,0.066528
quality_max,-0.014707,-0.284935,0.628587,0.294560,1.000000,-0.291470,0.206438,0.203001,0.187780,0.211232,...,0.306087,0.306087,-0.399548,-0.224842,-0.211464,-0.249401,0.226621,-0.475049,-0.373939,0.149633
quality_std,-0.039298,0.618199,-0.823734,-0.983021,-0.291470,1.000000,-0.611291,-0.597470,-0.523138,-0.773679,...,-0.261570,-0.261570,0.112873,0.113084,0.113887,0.145749,-0.141753,0.156776,0.101579,-0.060014
cum_reward_avg,0.008819,-0.416448,0.538676,0.615733,0.206438,-0.611291,1.000000,0.995714,0.945317,0.773693,...,0.093485,0.093485,-0.043420,0.022577,0.007990,-0.018156,0.082193,-0.040746,-0.001528,0.130301
cum_reward_max,0.004996,-0.402147,0.525389,0.602014,0.203001,-0.597470,0.995714,1.000000,0.961596,0.754861,...,0.085431,0.085431,-0.038677,0.030676,0.015317,-0.009432,0.074844,-0.032689,0.002849,0.126445
cum_reward_std,0.007599,-0.336589,0.463892,0.526560,0.187780,-0.523138,0.945317,0.961596,1.000000,0.661502,...,0.065849,0.065849,-0.022843,0.054104,0.036872,0.019841,0.056788,-0.008502,0.014264,0.100502
mom_reward_avg,0.001519,-0.532794,0.687860,0.774260,0.211232,-0.773679,0.773693,0.754861,0.661502,1.000000,...,0.188594,0.188594,-0.158871,-0.101068,-0.103590,-0.141259,0.142331,-0.181779,-0.113113,0.158887


## Weighted

In [142]:
fi = pd.read_csv('data/generated/fi_2020-2023.csv', index_col=0)
fi.rename(index={'Lower_limit': 'Param 1', 'Upper_limit': 'Param 2'}, inplace=True)
fi = fi / fi.max()
fi = fi.to_dict()['importance']
fi

{'HR Usage Rate': 1.0,
 'TCH Blocking Rate, BH': 0.05306688095919128,
 'Number of Available\nTCH': 0.9145249190288962,
 'TCH Traffic (Erl), BH': 0.48950080102867455,
 'Param 1': 0.934843836886342,
 'Param 2': 0.8525678849265464}

In [143]:
get_weighted_mean = partial(get_weighted_aggregation, fi=fi, method=np.sum)

In [144]:
df_weighted = orig_df[['cell_id']].copy()

df_weighted[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_weighted_mean, x)))
df_weighted

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,0.051957,0.051957,0.853906,0.414207,1.110201,0.653979,0.046703,0.592298,3.708187,0.423733
1,22944,0.052733,0.052733,1.562574,0.469713,1.196860,0.733897,0.035022,0.721307,5.870352,0.049540
2,26335,0.004215,0.004215,0.830895,0.399285,1.064499,0.643096,0.001436,0.537781,3.547887,0.759306
3,26332,0.011661,0.011661,0.893978,0.402643,1.107987,0.638254,0.007173,0.527759,3.664790,0.394179
4,26336,0.053066,0.053066,0.921858,0.384771,0.930966,0.636741,0.053067,0.632632,3.883982,0.156101
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,0.060378,0.060378,0.586777,0.163779,0.358007,0.407592,0.053456,0.392734,2.423370,0.406949
990,5682,0.060061,0.060061,0.518345,0.178481,0.392277,0.431083,0.034257,0.416556,2.129213,1.435655
991,12772,0.183872,0.183872,0.535775,0.196814,0.502577,0.452802,0.032943,0.414797,2.299909,1.026943
992,12771,0.213819,0.213819,0.433903,0.069349,0.143001,0.270271,0.131244,0.264042,1.760948,0.358493


In [145]:
merged = rewards.merge(df_weighted, left_on='cell_id', right_on='cell_id', how='inner')
merged.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,-0.061154,-0.061154,0.032188,-0.001288,-0.007983,0.003531,-0.050334,-0.000948,0.026429,-0.060271
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,-0.187910,-0.187910,0.105710,0.106913,0.101999,0.117565,-0.188916,0.131229,0.144316,0.050383
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,0.111313,0.111313,-0.217276,-0.183549,-0.181030,-0.195419,0.128911,-0.247091,-0.224413,-0.011932
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,0.019097,0.019097,-0.061715,-0.077037,-0.072327,-0.079798,0.031121,-0.074804,-0.080048,-0.012691
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,0.143882,0.143882,-0.430963,-0.275066,-0.268045,-0.298840,0.131997,-0.520536,-0.423233,0.054123
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,-0.014255,-0.014255,0.055408,0.063194,0.058380,0.065233,-0.035668,0.062384,0.072268,0.010462
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,-0.071164,-0.071164,-0.018869,0.020520,0.012938,0.022321,-0.015756,0.011074,-0.008143,0.008945
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,-0.116757,-0.116757,0.025052,0.064119,0.047400,0.068904,-0.054562,0.049643,0.033408,-0.000886
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,-0.182143,-0.182143,0.097127,0.130780,0.097932,0.140952,-0.120771,0.098292,0.102074,-0.026260
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,-0.031121,-0.031121,-0.073488,-0.038037,-0.038064,-0.036322,0.027264,-0.048523,-0.061438,0.007449


## Other funcs

In [146]:
orig_df

,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test,cell_id
0,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",25771
1,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",22944
2,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",26335
3,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",26332
4,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",26336
...,...,...,...,...,...,...,...,...,...,...,...
989,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...",5683
990,"{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.05,...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ...","{""HR Usage Rate"": {""stattest_threshold"": 0.1, ..

In [147]:
df_weighted = orig_df[['cell_id']].copy()

df_weighted[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_weighted_thresh_sum, x)))
rewards.merge(df_weighted, left_on='cell_id', right_on='cell_id', how='inner').corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,-0.002563,-0.002563,0.026872,0.007516,-0.002629,-0.010432,0.006406,-0.007972,0.025433,-0.057433
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,-0.653510,-0.653510,0.099011,0.057224,0.089327,0.126692,-0.526603,0.115862,0.139295,-0.052614
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,0.468463,0.468463,-0.259121,-0.150117,-0.180873,-0.178103,0.372056,-0.220031,-0.243119,0.166351
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,0.275802,0.275802,-0.086665,-0.057692,-0.073549,-0.080717,0.172363,-0.068739,-0.083771,0.142302
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,0.313838,0.313838,-0.393508,-0.221369,-0.294765,-0.294992,0.251381,-0.453909,-0.431314,0.108123
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,-0.291986,-0.291986,0.092411,0.055491,0.058775,0.064493,-0.195430,0.048431,0.077750,-0.178503
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,0.120458,0.120458,-0.078567,-0.023672,0.005064,0.028980,0.100122,0.041435,-0.017151,0.187819
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,0.089253,0.089253,-0.041244,-0.006576,0.036672,0.085396,0.072952,0.090143,0.022588,0.162934
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,0.011943,0.011943,0.027880,0.023567,0.084054,0.172061,0.013392,0.149329,0.088998,0.060395
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,0.147133,0.147133,-0.131456,-0.055309,-0.040133,-0.034952,0.118969,-0.018074,-0.071583,0.200328


In [148]:
df_weighted

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,3.166667,3.166667,1.166667,0.333333,1.833333,1.000000,1.333333,0.833333,6.333333,3.166667
1,22944,3.166667,3.166667,2.500000,0.666667,2.000000,1.000000,1.000000,1.000000,11.000000,3.000000
2,26335,0.000000,0.000000,1.166667,0.333333,1.833333,0.833333,0.000000,0.833333,6.166667,5.500000
3,26332,0.166667,0.166667,1.500000,0.333333,1.833333,0.833333,0.000000,0.666667,6.500000,3.166667
4,26336,3.166667,3.166667,1.500000,0.333333,1.333333,0.833333,1.500000,0.833333,6.833333,3.000000
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,3.166667,3.166667,0.833333,0.000000,0.166667,0.333333,1.500000,0.166667,4.000000,2.666667
990,5682,3.166667,3.166667,0.833333,0.000000,0.166667,0.333333,1.000000,0.166667,3.666667,8.333333
991,12772,0.500000,0.500000,0.666667,0.000000,0.500000,0.333333,0.000000,0.333333,4.166667,5.666667
992,12771,3.666667,3.666667,0.500000,0.000000,0.000000,0.000000,1.500000,0.000000,3.000000,4.333333


## Adaptive threshold

In [149]:
df_thresh = orig_df[['cell_id']].copy()

df_thresh[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_thresh_sum, x)))
df_thresh

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,3.166667,3.166667,1.166667,0.333333,1.833333,1.000000,1.333333,0.833333,6.333333,3.166667
1,22944,3.166667,3.166667,2.500000,0.666667,2.000000,1.000000,1.000000,1.000000,11.000000,3.000000
2,26335,0.000000,0.000000,1.166667,0.333333,1.833333,0.833333,0.000000,0.833333,6.166667,5.500000
3,26332,0.166667,0.166667,1.500000,0.333333,1.833333,0.833333,0.000000,0.666667,6.500000,3.166667
4,26336,3.166667,3.166667,1.500000,0.333333,1.333333,0.833333,1.500000,0.833333,6.833333,3.000000
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,3.166667,3.166667,0.833333,0.000000,0.166667,0.333333,1.500000,0.166667,4.000000,2.666667
990,5682,3.166667,3.166667,0.833333,0.000000,0.166667,0.333333,1.000000,0.166667,3.666667,8.333333
991,12772,0.500000,0.500000,0.666667,0.000000,0.500000,0.333333,0.000000,0.333333,4.166667,5.666667
992,12771,3.666667,3.666667,0.500000,0.000000,0.000000,0.000000,1.500000,0.000000,3.000000,4.333333


In [150]:
merged_thresh = rewards.merge(df_thresh, left_on='cell_id', right_on='cell_id', how='inner')
merged_thresh.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,-0.002563,-0.002563,0.026872,0.007516,-0.002629,-0.010432,0.006406,-0.007972,0.025433,-0.057433
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,-0.653510,-0.653510,0.099011,0.057224,0.089327,0.126692,-0.526603,0.115862,0.139295,-0.052614
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,0.468463,0.468463,-0.259121,-0.150117,-0.180873,-0.178103,0.372056,-0.220031,-0.243119,0.166351
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,0.275802,0.275802,-0.086665,-0.057692,-0.073549,-0.080717,0.172363,-0.068739,-0.083771,0.142302
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,0.313838,0.313838,-0.393508,-0.221369,-0.294765,-0.294992,0.251381,-0.453909,-0.431314,0.108123
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,-0.291986,-0.291986,0.092411,0.055491,0.058775,0.064493,-0.195430,0.048431,0.077750,-0.178503
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,0.120458,0.120458,-0.078567,-0.023672,0.005064,0.028980,0.100122,0.041435,-0.017151,0.187819
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,0.089253,0.089253,-0.041244,-0.006576,0.036672,0.085396,0.072952,0.090143,0.022588,0.162934
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,0.011943,0.011943,0.027880,0.023567,0.084054,0.172061,0.013392,0.149329,0.088998,0.060395
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,0.147133,0.147133,-0.131456,-0.055309,-0.040133,-0.034952,0.118969,-0.018074,-0.071583,0.200328


## Adaptive weighted threshold

In [151]:
df_weighted_thresh = orig_df[['cell_id']].copy()

df_weighted_thresh[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_weighted_thresh_sum, x)))
df_weighted_thresh

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,3.166667,3.166667,1.166667,0.333333,1.833333,1.000000,1.333333,0.833333,6.333333,3.166667
1,22944,3.166667,3.166667,2.500000,0.666667,2.000000,1.000000,1.000000,1.000000,11.000000,3.000000
2,26335,0.000000,0.000000,1.166667,0.333333,1.833333,0.833333,0.000000,0.833333,6.166667,5.500000
3,26332,0.166667,0.166667,1.500000,0.333333,1.833333,0.833333,0.000000,0.666667,6.500000,3.166667
4,26336,3.166667,3.166667,1.500000,0.333333,1.333333,0.833333,1.500000,0.833333,6.833333,3.000000
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,3.166667,3.166667,0.833333,0.000000,0.166667,0.333333,1.500000,0.166667,4.000000,2.666667
990,5682,3.166667,3.166667,0.833333,0.000000,0.166667,0.333333,1.000000,0.166667,3.666667,8.333333
991,12772,0.500000,0.500000,0.666667,0.000000,0.500000,0.333333,0.000000,0.333333,4.166667,5.666667
992,12771,3.666667,3.666667,0.500000,0.000000,0.000000,0.000000,1.500000,0.000000,3.000000,4.333333


In [152]:
df_weighted_thresh = rewards.merge(df_weighted_thresh, left_on='cell_id', right_on='cell_id', how='inner')
df_weighted_thresh.corr()

,cell_id,drift_score,quality_avg,quality_min,quality_max,quality_std,cum_reward_avg,cum_reward_max,cum_reward_std,mom_reward_avg,...,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
cell_id,1.000000,-0.042054,0.036162,0.018046,-0.001332,-0.023063,0.000967,-0.002974,-0.010015,-0.015267,...,-0.002563,-0.002563,0.026872,0.007516,-0.002629,-0.010432,0.006406,-0.007972,0.025433,-0.057433
drift_score,-0.042054,1.000000,-0.637122,-0.609597,-0.289934,0.642976,-0.395991,-0.362477,-0.190325,-0.412066,...,-0.653510,-0.653510,0.099011,0.057224,0.089327,0.126692,-0.526603,0.115862,0.139295,-0.052614
quality_avg,0.036162,-0.637122,1.000000,0.689293,0.462465,-0.782348,0.615313,0.612029,0.471206,0.680864,...,0.468463,0.468463,-0.259121,-0.150117,-0.180873,-0.178103,0.372056,-0.220031,-0.243119,0.166351
quality_min,0.018046,-0.609597,0.689293,1.000000,0.197964,-0.955504,0.643323,0.588608,0.291974,0.660707,...,0.275802,0.275802,-0.086665,-0.057692,-0.073549,-0.080717,0.172363,-0.068739,-0.083771,0.142302
quality_max,-0.001332,-0.289934,0.462465,0.197964,1.000000,-0.190559,0.118900,0.117325,0.090388,0.110648,...,0.313838,0.313838,-0.393508,-0.221369,-0.294765,-0.294992,0.251381,-0.453909,-0.431314,0.108123
quality_std,-0.023063,0.642976,-0.782348,-0.955504,-0.190559,1.000000,-0.760596,-0.714535,-0.413002,-0.800001,...,-0.291986,-0.291986,0.092411,0.055491,0.058775,0.064493,-0.195430,0.048431,0.077750,-0.178503
cum_reward_avg,0.000967,-0.395991,0.615313,0.643323,0.118900,-0.760596,1.000000,0.963393,0.626701,0.936032,...,0.120458,0.120458,-0.078567,-0.023672,0.005064,0.028980,0.100122,0.041435,-0.017151,0.187819
cum_reward_max,-0.002974,-0.362477,0.612029,0.588608,0.117325,-0.714535,0.963393,1.000000,0.804616,0.884176,...,0.089253,0.089253,-0.041244,-0.006576,0.036672,0.085396,0.072952,0.090143,0.022588,0.162934
cum_reward_std,-0.010015,-0.190325,0.471206,0.291974,0.090388,-0.413002,0.626701,0.804616,1.000000,0.535230,...,0.011943,0.011943,0.027880,0.023567,0.084054,0.172061,0.013392,0.149329,0.088998,0.060395
mom_reward_avg,-0.015267,-0.412066,0.680864,0.660707,0.110648,-0.800001,0.936032,0.884176,0.535230,1.000000,...,0.147133,0.147133,-0.131456,-0.055309,-0.040133,-0.034952,0.118969,-0.018074,-0.071583,0.200328


# Get data for learning

In [185]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split
from joblib import dump, load
import os.path as osp
import numpy as np

In [153]:
def get_by_feature_score(x: str) -> Dict[str, any]:
    js = json.loads(x)

    return {f: js[f]['drift_score'] for f,v in js.items()}

In [195]:
def get_models_results(df: pd.DataFrame,
                       rewards_df: pd.DataFrame,
                       features_l: List[str],
                       target: str,
                       stat_ts: List[str],
                       save_path: str) -> pd.DataFrame:
    df_4_train = df[['cell_id']].copy()
    df_4_train[stat_ts] = orig_df[stat_ts].apply(lambda x: list(map(get_by_feature_score, x)))

    results = []

    for st in stat_ts:
        train_df = pd.DataFrame(df_4_train[st].tolist(), index=df_4_train['cell_id'])
        train_df = train_df.merge(rewards_df[['cell_id', target]], left_on='cell_id', right_on='cell_id', how='inner')

        X_train, X_test, y_train, y_test = train_test_split(train_df[features_l], train_df[target], test_size=0.3, random_state=3407)

        # Linear Regression
        model_lr = LinearRegression()
        model_lr.fit(X_train, y_train)
        lr_train_score = model_lr.score(X_train, y_train)
        lr_test_score = model_lr.score(X_test, y_test)
        # dumping model
        lr_path = osp.join(save_path, f'lr_{st}.joblib')
        dump(model_lr, lr_path)

        # Polynomial
        transformer = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
        X_train_polynom = transformer.fit_transform(X_train)
        X_test_polynom = transformer.transform(X_test)
        model_pol_lr = LinearRegression().fit(X_train_polynom, y_train)
        pr_train_score = model_pol_lr.score(X_train_polynom, y_train)
        pr_test_score = model_pol_lr.score(X_test_polynom, y_test)
        # dumping model
        pol_lr_path = osp.join(save_path, f'pol_lr_{st}.joblib')
        trans_path = osp.join(save_path, f'transformer_{st}.joblib')
        dump(model_pol_lr, pol_lr_path)
        dump(transformer, trans_path)


        # save results
        results.append((st,
                        lr_train_score, lr_test_score,
                        pr_train_score, pr_test_score,
                        lr_path,
                        pol_lr_path,
                        trans_path))

    res_df = pd.DataFrame(results, columns=['stat_test',
                                            'lr_train_score', 'lr_test_score',
                                            'pol_lr_train_score', 'pol_lr_test_score',
                                            'lr_path',
                                            'pol_lr_path',
                                            'trans_path'])

    return res_df

In [159]:
features = list(fi.keys())
features

['HR Usage Rate',
 'TCH Blocking Rate, BH',
 'Number of Available\nTCH',
 'TCH Traffic (Erl), BH',
 'Param 1',
 'Param 2']

In [160]:
rewards.columns


Index(['cell_id', 'drift_score', 'quality_avg', 'quality_min', 'quality_max',
       'quality_std', 'cum_reward_avg', 'cum_reward_max', 'cum_reward_std',
       'mom_reward_avg', 'mom_reward_min', 'mom_reward_max', 'mom_reward_std'],
      dtype='object')

In [166]:
label = 'quality_avg'

In [197]:
path = 'data/generated/models/one_drift_0'
get_models_results(orig_df, rewards, features, label, stat_tests, path)

,stat_test,lr_train_score,lr_test_score,pol_lr_train_score,pol_lr_test_score,lr_path,pol_lr_path,trans_path
0,default,0.688865,0.731751,0.801007,0.747313,data/generated/models/one_drift_0/lr_default.j...,data/generated/models/one_drift_0/pol_lr_defau...,data/generated/models/one_drift_0/transformer_...
1,ks,0.033693,-0.248335,0.034097,-0.277876,data/generated/models/one_drift_0/lr_ks.joblib,data/generated/models/one_drift_0/pol_lr_ks.jo...,data/generated/models/one_drift_0/transformer_...
2,wasserstein,0.688865,0.731751,0.801007,0.747313,data/generated/models/one_drift_0/lr_wasserste...,data/generated/models/one_drift_0/pol_lr_wasse...,data/generated/models/one_drift_0/transformer_...
3,kl_div,0.634255,0.712781,0.660252,0.619108,data/generated/models/one_drift_0/lr_kl_div.jo...,data/generated/models/one_drift_0/pol_lr_kl_di...,data/generated/models/one_drift_0/transformer_...
4,psi,0.656159,0.736471,0.682675,0.686402,data/generated/models/one_drift_0/lr_psi.joblib,data/generated/models/one_drift_0/pol_lr_psi.j...,data/generated/models/one_drift_0/transformer_...
5,jensenshannon,0.688177,0.788680,0.724999,0.780046,data/generated/models/one_drift_0/lr_jensensha...,data/generated/models/one_drift_0/pol_lr_jense...,data/generated/models/one_drift_0/transformer_...
6,cramer_von_mises,0.035772,-0.016107,0.074289,-0.002753,data/generated/models/one_drift_0/lr_cramer_vo...,data/generated/models/one_drift_0/pol_lr_crame...,data/generated/models/one_drift_0/transformer_...
7,hellinger,0.590323,0.652020,0.608425,0.657898,data/generated/models/one_drift_0/lr_hellinger...,data/generated/models/one_drift_0/pol_lr_helli...,data/generated/models/one_drift_0/transformer_...
8,ed,0.755561,0.698535,0.772892,0.714460,data/generated/models/one_drift_0/lr_ed.joblib,data/generated/models/one_drift_0/pol_lr_ed.jo...,data/generated/models/one_drift_0/transformer_...
9,t_test,0.145621,0.010137,0.159913,-0.187565,data/generated/models/one_drift_0/lr_t_test.jo...,data/generated/models/one_drift_0/pol_lr_t_tes...,data/generated/models/one_drift_0/transformer_...


## Getting one drift all run

In [199]:
fi

{'HR Usage Rate': 1.0,
 'TCH Blocking Rate, BH': 0.05306688095919128,
 'Number of Available\nTCH': 0.9145249190288962,
 'TCH Traffic (Erl), BH': 0.48950080102867455,
 'Param 1': 0.934843836886342,
 'Param 2': 0.8525678849265464}

In [200]:
stat_tests

['default',
 'ks',
 'wasserstein',
 'kl_div',
 'psi',
 'jensenshannon',
 'cramer_von_mises',
 'hellinger',
 'ed',
 't_test']

In [222]:
datasets_list = [('not-sampled_no-ref-window', 'data/generated/drift/by_cell_agent/run_7/by_train_regressive__sampled-drift-None_no-window_.csv'),
             ('sampled_no-ref-window', 'data/generated/drift/by_cell_agent/run_8/by_train_regressive__sampled-drift-1000_no-window_.csv'),
             ('not-sampled_ref-window-1k', 'data/generated/drift/by_cell_agent/run_9/by_train_regressive_sampled_ref_sampled_drift_None_window_1k_.csv'),
             ('sampled_ref-window-1k', 'data/generated/drift/by_cell_agent/run_10/by_train_regressive_sampled_ref_sampled_drift_1000_window_1k_.csv')]
path = 'data/generated/models/one_drift_1'
label = 'quality_avg'
features = ['HR Usage Rate',
 'TCH Blocking Rate, BH',
 'Number of Available\nTCH',
 'TCH Traffic (Erl), BH',
 'Param 1',
 'Param 2']
one_drift_res = pd.DataFrame()

for dataset_name, dataset_path in datasets_list:
    dataset = pd.read_csv(dataset_path, index_col=0)

    for form_data_func in [get_regress_score, get_weighted_mean, get_thresh_sum, get_weighted_thresh_sum]:
        data_type_name = form_data_func.__name__

        regress_df = dataset[['cell_id']].copy()
        regress_df[stat_tests] = dataset[stat_tests].apply(lambda x: list(map(form_data_func, x)))
        # creating path to save data
        saving_path = osp.join(path, dataset_name, data_type_name)
        Path(saving_path).mkdir(exist_ok=True, parents=True)

        res = get_models_results(orig_df, rewards, features, label, stat_tests, saving_path)
        res['dataset_name'] = [dataset_name] * len(res)
        res['data_type_name'] = [data_type_name] * len(res)

        one_drift_res = pd.concat([one_drift_res, res])

one_drift_res.to_csv(osp.join(path, 'one_drift_res.csv'))
one_drift_res

,stat_test,lr_train_score,lr_test_score,pol_lr_train_score,pol_lr_test_score,lr_path,pol_lr_path,trans_path,dataset_name,data_type_name
0,default,0.688865,0.731751,0.801007,0.747313,data/generated/models/one_drift_1/not-sampled_...,data/generated/models/one_drift_1/not-sampled_...,data/generated/models/one_drift_1/not-sampled_...,not-sampled_no-ref-window,get_regress_score
1,ks,0.033693,-0.248335,0.034097,-0.277876,data/generated/models/one_drift_1/not-sampled_...,data/generated/models/one_drift_1/not-sampled_...,data/generated/models/one_drift_1/not-sampled_...,not-sampled_no-ref-window,get_regress_score
2,wasserstein,0.688865,0.731751,0.801007,0.747313,data/generated/models/one_drift_1/not-sampled_...,data/generated/models/one_drift_1/not-sampled_...,data/generated/models/one_drift_1/not-sampled_...,not-sampled_no-ref-window,get_regress_score
3,kl_div,0.634255,0.712781,0.660252,0.619108,data/generated/models/one_drift_1/not-sampled_...,data/generated/models/one_drift_1/not-sampled_...,data/generated/models/one_drift_1/not-sampled_...,not-sampled_no-ref-window,get_regress_score
4,psi,0.656159,0.736471,0.682675,0.686402,data/generated/models/one_drift_1/not-sampled_...,data/generated/models/one_drift_1/not-sampled_...,data/generated/models/one_drift_1/not-sampled_...,not-sampled_no-ref-window,get_regress_score
...,...,...,...,...,...,...,...,...,...,...
5,jensenshannon,0.688177,0.788680,0.724999,0.780046,data/generated/models/one_drift_1/sampled_ref-...,data/generated/models/one_drift_1/sampled_ref-...,data/generated/models/one_drift_1/sampled_ref-...,sampled_ref-window-1k,get_weighted_thresh_sum
6,cramer_von_mises,0.035772,-0.016107,0.074289,-0.002753,data/generated/models/one_drift_1/sampled_ref-...,data/generated/models/one_drift_1/sampled_ref-...,data/generated/models/one_drift_1/sampled_ref-...,sampled_ref-window-1k,get_weighted_thresh_sum
7,hellinger,0.590323,0.652020,0.608425,0.657898,data/generated/models/one_drift_1/sampled_ref-...,data/generated/models/one_drift_1/sampled_ref-...,data/generated/models/one_drift_1/sampled_ref-...,sampled_ref-window-1k,get_weighted_thresh_sum
8,ed,0.755561,0.698535,0.772892,0.714460,data/generated/models/one_drift_1/sampled_ref-...,data/generated/models/one_drift_1/sampled_ref-...,data/generated/models/one_drift_1/sampled_ref-...,sampled_ref-window-1k,get_weighted_thresh_sum


In [219]:
get_weighted_mean.__name__

AttributeError: 'functools.partial' object has no attribute '__name__'

## Getting multi test

In [223]:
def get_models_poly_drift_results(df: pd.DataFrame,
                       rewards_df: pd.DataFrame,
                       features_l: List[str],
                       target: str,
                       stat_ts: List[str],
                       save_path: str) -> pd.DataFrame:
    df_4_train = df[['cell_id']].copy()
    df_4_train[stat_ts] = orig_df[stat_ts].apply(lambda x: list(map(get_by_feature_score, x)))

    results = []
    train_df = pd.DataFrame(index=df_4_train['cell_id'])

    for st in stat_ts:
        temp_df = pd.DataFrame(df_4_train[st].tolist(), index=df_4_train['cell_id'])
        train_df = pd.merge(train_df, temp_df, left_on='cell_id', right_on='cell_id', how='inner', suffixes=('', '_' + st))

    train_df = train_df.merge(rewards_df[['cell_id', target]], left_on='cell_id', right_on='cell_id', how='inner')
    print(train_df)
    X_train, X_test, y_train, y_test = train_test_split(train_df[features_l], train_df[target], test_size=0.3, random_state=3407)

    # Linear Regression
    model_lr = LinearRegression()
    model_lr.fit(X_train, y_train)
    lr_train_score = model_lr.score(X_train, y_train)
    lr_test_score = model_lr.score(X_test, y_test)
    # dumping model
    lr_path = osp.join(save_path, f'lr_all_stats.joblib')
    dump(model_lr, lr_path)

    # Polynomial
    transformer = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
    X_train_polynom = transformer.fit_transform(X_train)
    X_test_polynom = transformer.transform(X_test)
    model_pol_lr = LinearRegression().fit(X_train_polynom, y_train)
    pr_train_score = model_pol_lr.score(X_train_polynom, y_train)
    pr_test_score = model_pol_lr.score(X_test_polynom, y_test)
    # dumping model
    pol_lr_path = osp.join(save_path, f'pol_lr_all_stats.joblib')
    trans_path = osp.join(save_path, f'transformer_all_stats.joblib')
    dump(model_pol_lr, pol_lr_path)
    dump(transformer, trans_path)

    # Cross Decomposition
    reg = PLSRegression(n_components=len(features_l))
    reg.fit(X_train, y_train.to_numpy().ravel())
    plsr_train_score = reg.score(X_train, y_train.to_numpy().ravel())
    plsr_test_score = reg.score(X_test, y_test.to_numpy().ravel())
    # dumping model
    plsr_path = osp.join(save_path, f'plsr_all_stats.joblib')
    dump(reg, plsr_path)

    # save results
    results.append((lr_train_score, lr_test_score,
                    pr_train_score, pr_test_score,
                    plsr_train_score, plsr_test_score,
                    lr_path,
                    pol_lr_path,
                    trans_path,
                    plsr_path))

    res_df = pd.DataFrame(results, columns=['lr_train_score', 'lr_test_score',
                                            'pol_lr_train_score', 'pol_lr_test_score',
                                            'plsr_train_score', 'plsr_test_score',
                                            'lr_path',
                                            'pol_lr_path',
                                            'trans_path',
                                            'plsr_path'])

    return res_df

In [194]:
path = '/home/rid/Projects/Study/Magister/Dyploma/Bachelor/data/generated/models/poly_drift_0'
get_models_poly_drift_results(orig_df, rewards, features, label, stat_tests, path)

     cell_id  HR Usage Rate  Number of Available\nTCH   Param 1   Param 2  \
0      25771       1.197629                  0.784173  1.050900  0.926529   
1      22944       1.297928                  2.191368  1.613654  1.655267   
2      26335       0.907132                  0.785231  0.869361  1.265191   
3      26332       0.537904                  0.786818  1.401794  1.425661   
4      26336       1.061184                  1.181181  0.936464  1.021152   
..       ...            ...                       ...       ...       ...   
989     5683       0.982010                  0.773861  0.785553  0.863654   
990     5682       0.841023                  0.779998  0.747635  0.817129   
991    12772       1.246695                  0.786818  1.030905  0.940693   
992    12771       0.742420                  1.032760  0.889983  1.003829   
993    12773       0.668629                  0.696427  0.798435  0.878093   

     TCH Blocking Rate, BH  TCH Traffic (Erl), BH  HR Usage Rate_ks  \
0   

,lr_train_score,lr_test_score,pol_lr_train_score,pol_lr_test_score,plsr_train_score,plsr_test_score,lr_path,pol_lr_path,trans_path,plsr_path
0,0.688865,0.731751,0.801007,0.747313,0.688865,0.731751,/home/rid/Projects/Study/Magister/Dyploma/Bach...,/home/rid/Projects/Study/Magister/Dyploma/Bach...,/home/rid/Projects/Study/Magister/Dyploma/Bach...,/home/rid/Projects/Study/Magister/Dyploma/Bach...


### poly drift all run

In [224]:
datasets_list = [('not-sampled_no-ref-window', 'data/generated/drift/by_cell_agent/run_7/by_train_regressive__sampled-drift-None_no-window_.csv'),
             ('sampled_no-ref-window', 'data/generated/drift/by_cell_agent/run_8/by_train_regressive__sampled-drift-1000_no-window_.csv'),
             ('not-sampled_ref-window-1k', 'data/generated/drift/by_cell_agent/run_9/by_train_regressive_sampled_ref_sampled_drift_None_window_1k_.csv'),
             ('sampled_ref-window-1k', 'data/generated/drift/by_cell_agent/run_10/by_train_regressive_sampled_ref_sampled_drift_1000_window_1k_.csv')]
path = 'data/generated/models/poly_drift_1'
label = 'quality_avg'
features = ['HR Usage Rate',
 'TCH Blocking Rate, BH',
 'Number of Available\nTCH',
 'TCH Traffic (Erl), BH',
 'Param 1',
 'Param 2']
poly_drift_res = pd.DataFrame()

for dataset_name, dataset_path in datasets_list:
    dataset = pd.read_csv(dataset_path, index_col=0)

    for form_data_func in [get_regress_score, get_weighted_mean, get_thresh_sum, get_weighted_thresh_sum]:
        data_type_name = form_data_func.__name__

        regress_df = dataset[['cell_id']].copy()
        regress_df[stat_tests] = dataset[stat_tests].apply(lambda x: list(map(form_data_func, x)))
        # creating path to save data
        saving_path = osp.join(path, dataset_name, data_type_name)
        Path(saving_path).mkdir(exist_ok=True, parents=True)

        res = get_models_poly_drift_results(orig_df, rewards, features, label, stat_tests, saving_path)
        res['dataset_name'] = [dataset_name] * len(res)
        res['data_type_name'] = [data_type_name] * len(res)

        poly_drift_res = pd.concat([poly_drift_res, res])

poly_drift_res.to_csv(osp.join(path, 'poly_drift_res.csv'))
poly_drift_res

     cell_id  HR Usage Rate  Number of Available\nTCH   Param 1   Param 2  \
0      25771       1.197629                  0.784173  1.050900  0.926529   
1      22944       1.297928                  2.191368  1.613654  1.655267   
2      26335       0.907132                  0.785231  0.869361  1.265191   
3      26332       0.537904                  0.786818  1.401794  1.425661   
4      26336       1.061184                  1.181181  0.936464  1.021152   
..       ...            ...                       ...       ...       ...   
989     5683       0.982010                  0.773861  0.785553  0.863654   
990     5682       0.841023                  0.779998  0.747635  0.817129   
991    12772       1.246695                  0.786818  1.030905  0.940693   
992    12771       0.742420                  1.032760  0.889983  1.003829   
993    12773       0.668629                  0.696427  0.798435  0.878093   

     TCH Blocking Rate, BH  TCH Traffic (Erl), BH  HR Usage Rate_ks  \
0   

,lr_train_score,lr_test_score,pol_lr_train_score,pol_lr_test_score,plsr_train_score,plsr_test_score,lr_path,pol_lr_path,trans_path,plsr_path,dataset_name,data_type_name
0,0.688865,0.731751,0.801007,0.747313,0.688865,0.731751,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,not-sampled_no-ref-window,get_regress_score
0,0.688865,0.731751,0.801007,0.747313,0.688865,0.731751,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,not-sampled_no-ref-window,get_weighted_mean
0,0.688865,0.731751,0.801007,0.747313,0.688865,0.731751,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,not-sampled_no-ref-window,get_thresh_sum
0,0.688865,0.731751,0.801007,0.747313,0.688865,0.731751,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,not-sampled_no-ref-window,get_weighted_thresh_sum
0,0.688865,0.731751,0.801007,0.747313,0.688865,0.731751,data/generated/models/poly_drift_1/sampled_no-...,data/generated/models/poly_drift_1/sampled_no-...,data/generated/models/poly_drift_1/sampled_no-...,data/generated/models/poly_drift_1/sampled_no-...,sampled_no-ref-window,get_regress_score
0,0.688865,0.731751,0.801007,0.747313,0.688865,0.731751,data/generated/models/poly_drift_1/sampled_no-...,data/generated/models/poly_drift_1/sampled_no-...,data/generated/models/poly_drift_1/sampled_no-...,data/generated/models/poly_drift_1/sampled_no-...,sampled_no-ref-window,get_weighted_mean
0,0.688865,0.731751,0.801007,0.747313,0.688865,0.731751,data/generated/models/poly_drift_1/sampled_no-...,data/generated/models/poly_drift_1/sampled_no-...,data/generated/models/poly_drift_1/sampled_no-...,data/generated/models/poly_drift_1/sampled_no-...,sampled_no-ref-window,get_thresh_sum
0,0.688865,0.731751,0.801007,0.747313,0.688865,0.731751,data/generated/models/poly_drift_1/sampled_no-...,data/generated/models/poly_drift_1/sampled_no-...,data/generated/models/poly_drift_1/sampled_no-...,data/generated/models/poly_drift_1/sampled_no-...,sampled_no-ref-window,get_weighted_thresh_sum
0,0.688865,0.731751,0.801007,0.747313,0.688865,0.731751,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,not-sampled_ref-window-1k,get_regress_score
0,0.688865,0.731751,0.801007,0.747313,0.688865,0.731751,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,data/generated/models/poly_drift_1/not-sampled...,not-sampled_ref-window-1k,get_weighted_mean


In [172]:
df_to_train = orig_df[['cell_id']].copy()

df_to_train[stat_tests] = orig_df[stat_tests].apply(lambda x: list(map(get_by_feature_score, x)))
df_to_train

,cell_id,default,ks,wasserstein,kl_div,psi,jensenshannon,cramer_von_mises,hellinger,ed,t_test
0,25771,"{'HR Usage Rate': 1.1976287701332842, 'Number ...","{'HR Usage Rate': 7.591622950770644e-104, 'Num...","{'HR Usage Rate': 1.1976287701332842, 'Number ...","{'HR Usage Rate': 3.7062426735260177, 'Number ...","{'HR Usage Rate': 4.86724178342445, 'Number of...","{'HR Usage Rate': 0.575628726830956, 'Number o...","{'HR Usage Rate': 4.905048662462441e-06, 'Numb...","{'HR Usage Rate': 0.6574633087573316, 'Number ...","{'HR Usage Rate': 6.065705893467136, 'Number o...","{'HR Usage Rate': 1.0419153009893417e-68, 'Num..."
1,22944,"{'HR Usage Rate': 1.2979277703511152, 'Number ...","{'HR Usage Rate': 1.1334687345106701e-135, 'Nu...","{'HR Usage Rate': 1.2979277703511152, 'Number ...","{'HR Usage Rate': 4.163990459381986, 'Number o...","{'HR Usage Rate': 5.622165134851706, 'Number o...","{'HR Usage Rate': 0.6279968022047079, 'Number ...","{'HR Usage Rate': 4.9088264401087045e-06, 'Num...","{'HR Usage Rate': 0.7136675083131901, 'Number ...","{'HR Usage Rate': 6.630028523223908, 'Number o...","{'HR Usage Rate': 2.2704136824497126e-80, 'Num..."
2,26335,"{'HR Usage Rate': 0.9071320123769095, 'Number ...","{'HR Usage Rate': 1.2653943129504837e-64, 'Num...","{'HR Usage Rate': 0.9071320123769095, 'Number ...","{'HR Usage Rate': 2.780294497033895, 'Number o...","{'HR Usage Rate': 4.074188776713164, 'Number o...","{'HR Usage Rate': 0.5595246271064998, 'Number ...","{'HR Usage Rate': 5.0004824249105795e-06, 'Num...","{'HR Usage Rate': 0.6009453204497797, 'Number ...","{'HR Usage Rate': 4.72124461485425, 'Number of...","{'HR Usage Rate': 1.8510901624030256e-36, 'Num..."
3,26332,"{'HR Usage Rate': 0.5379044436322102, 'Number ...","{'HR Usage Rate': 6.081317437220293e-21, 'Numb...","{'HR Usage Rate': 0.5379044436322102, 'Number ...","{'HR Usage Rate': 2.488766830623767, 'Number o...","{'HR Usage Rate': 3.1950646518384946, 'Number ...","{'HR Usage Rate': 0.44950735434077754, 'Number...","{'HR Usage Rate': 4.997305700693566e-06, 'Numb...","{'HR Usage Rate': 0.5019637836597516, 'Number ...","{'HR Usage Rate': 2.725551669592239, 'Number o...","{'HR Usage Rate': 0.0027064095018628203, 'Numb..."
4,26336,"{'HR Usage Rate': 1.061184487440827, 'Number o...","{'HR Usage Rate': 1.5027311605770971e-80, 'Num...","{'HR Usage Rate': 1.061184487440827, 'Number o...","{'HR Usage Rate': 2.5136201786415784, 'Number ...","{'HR Usage Rate': 3.7042953544828925, 'Number ...","{'HR Usage Rate': 0.5374976455558951, 'Number ...","{'HR Usage Rate': 5.001573155971606e-06, 'Numb...","{'HR Usage Rate': 0.5721575798050974, 'Number ...","{'HR Usage Rate': 5.416631967292511, 'Number o...","{'HR Usage Rate': 1.3959248434144886e-52, 'Num..."
...,...,...,...,...,...,...,...,...,...,...,...
989,5683,"{'HR Usage Rate': 0.9820104306067731, 'Number ...","{'HR Usage Rate': 2.5903946707991135e-76, 'Num...","{'HR Usage Rate': 0.9820104306067731, 'Number ...","{'HR Usage Rate': 4.388459114030215, 'Number o...","{'HR Usage Rate': 7.161338658841928, 'Number o...","{'HR Usage Rate': 0.7334613985642259, 'Number ...","{'HR Usage Rate': 5.669057461799021e-06, 'Numb...","{'HR Usage Rate': 0.7983310936310104, 'Number ...","{'HR Usage Rate': 5.008225431708757, 'Number o...","{'HR Usage Rate': 3.2955950015159177e-19, 'Num..."
990,5682,"{'HR Usage Rate': 0.8410234443432856, 'Number ...","{'HR Usage Rate': 1.874644384303983e-40, 'Numb...","{'HR Usage Rate': 0.8410234443432856, 'Number ...","{'HR Usage Rate': 5.097381368509053, 'Number o...","{'HR Usage Rate': 8.000810785035734, 'Number o...","{'HR Usage Rate': 0.7427547383910575, 'Number ...","{'HR Usage Rate': 5.667761503902469e-06, 'Numb...","{'HR Usage Rate': 0.7951676130364934, 'Number ...","{'HR Usage Rate': 4.129336781319746, 'Number o...","{'HR Usage Rate': 0.3640913158645145, 'Number ..."
991,12772,"{'HR Usage Rate': 1.2466948151441566, 'Number ...","{'HR Usage Rate': 8.341284159686772e-69, 'Numb...","{'HR Usage Rate': 1.2466948151441566, 'Number

In [26]:
pd.DataFrame(df_to_train['default'].tolist(), index=df_to_train['cell_id'])

,HR Usage Rate,Number of Available\nTCH,Param 1,Param 2,"TCH Blocking Rate, BH","TCH Traffic (Erl), BH"
cell_id,,,,,,
25771,1.197629,0.784173,1.050900,0.926529,0.080808,0.584263
22944,1.297928,2.191368,1.613654,1.655267,0.082576,1.672209
26335,0.907132,0.785231,0.869361,1.265191,0.106585,0.566432
26332,0.537904,0.786818,1.401794,1.425661,0.117485,0.561885
26336,1.061184,1.181181,0.936464,1.021152,0.071666,0.811645
...,...,...,...,...,...,...
5683,0.982010,0.773861,0.785553,0.863654,0.077324,0.661153
5682,0.841023,0.779998,0.747635,0.817129,0.079045,0.649416
12772,1.246695,0.786818,1.030905,0.940693,0.165583,1.135882


In [181]:

tmp_res = pd.DataFrame(index=df_to_train['cell_id'])

for st in stat_tests:
    tmp = pd.DataFrame(df_to_train[st].tolist(), index=df_to_train['cell_id'])
    tmp_res = pd.merge(tmp_res, tmp, left_on='cell_id', right_on='cell_id', how='inner', suffixes=('', '_' + st))

tmp_res

,HR Usage Rate,Number of Available\nTCH,Param 1,Param 2,"TCH Blocking Rate, BH","TCH Traffic (Erl), BH",HR Usage Rate_ks,Number of Available\nTCH_ks,Param 1_ks,Param 2_ks,...,Param 1_ed,Param 2_ed,"TCH Blocking Rate, BH_ed","TCH Traffic (Erl), BH_ed",HR Usage Rate_t_test,Number of Available\nTCH_t_test,Param 1_t_test,Param 2_t_test,"TCH Blocking Rate, BH_t_test","TCH Traffic (Erl), BH_t_test"
cell_id,,,,,,,,,,,,,,,,,,,,,
25771,1.197629,0.784173,1.050900,0.926529,0.080808,0.584263,7.591623e-104,1.879251e-41,3.556443e-121,2.629422e-100,...,5.010297,4.445901,0.079335,1.360775,1.041915e-68,5.148241e-05,9.564843e-14,2.781442e-04,0.492073,2.322765e-06
22944,1.297928,2.191368,1.613654,1.655267,0.082576,1.672209,1.133469e-135,7.362552e-253,8.256022e-156,5.855831e-174,...,7.224778,7.620695,0.086742,3.940322,2.270414e-80,8.993711e-220,1.550624e-107,7.918434e-124,0.227054,7.029083e-128
26335,0.907132,0.785231,0.869361,1.265191,0.106585,0.566432,1.265394e-64,1.879251e-41,1.559945e-101,8.308986e-132,...,4.045081,6.129369,0.265323,1.309287,1.851090e-36,5.499490e-05,1.706601e-01,8.111307e-46,0.663555,1.918995e-08
26332,0.537904,0.786818,1.401794,1.425661,0.117485,0.561885,6.081317e-21,1.879251e-41,0.000000e+00,0.000000e+00,...,6.480524,6.782690,0.214598,1.293051,2.706410e-03,6.069208e-05,1.988346e-93,1.499059e-96,0.130952,3.308649e-06
26336,1.061184,1.181181,0.936464,1.021152,0.071666,0.811645,1.502731e-80,2.742067e-140,1.217028e-108,8.522726e-112,...,4.426446,4.973958,0.050677,2.145887,1.395925e-52,7.211670e-55,1.442323e-04,3.078707e-12,0.349701,1.347800e-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5683,0.982010,0.773861,0.785553,0.863654,0.077324,0.661153,2.590395e-76,2.850837e-32,1.920628e-68,7.116561e-70,...,3.574599,4.120631,0.068662,1.597084,3.295595e-19,2.166344e-04,6.097183e-20,2.267497e-22,0.319129,5.772220e-06
5682,0.841023,0.779998,0.747635,0.817129,0.079045,0.649416,1.874644e-40,2.850837e-32,8.084292e-59,4.233801e-60,...,3.334335,3.834367,0.075792,1.583515,3.640913e-01,2.949286e-04,9.769606e-17,1.995662e-18,0.308480,3.903197e-02
12772,1.246695,0.786818,1.030905,0.940693,0.165583,1.135882,8.341284e-69,1.792384e-23,1.197157e-65,1.004326e-56,...,4.913287,4.528865,0.405727,3.039756,3.318201e-42,2.674240e-03,3.033114e-07,1.697265e-03,0.270951,8.943454e-27


## Test on linear regression

In [157]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split
from joblib import dump, load
import os.path as osp
import numpy as np

In [100]:
save_path = '/home/rid/Projects/Study/Magister/Dyploma/Bachelor/data/generated/models'

In [29]:
df_default = pd.DataFrame(df_to_train['default'].tolist(), index=df_to_train['cell_id'])
df_default = df_default.merge(rewards[['cell_id', 'quality_avg']], left_on='cell_id', right_on='cell_id', how='inner')
df_default

,cell_id,HR Usage Rate,Number of Available\nTCH,Param 1,Param 2,"TCH Blocking Rate, BH","TCH Traffic (Erl), BH",quality_avg
0,25771,1.197629,0.784173,1.050900,0.926529,0.080808,0.584263,0.986406
1,22944,1.297928,2.191368,1.613654,1.655267,0.082576,1.672209,0.983747
2,26335,0.907132,0.785231,0.869361,1.265191,0.106585,0.566432,0.986885
3,26332,0.537904,0.786818,1.401794,1.425661,0.117485,0.561885,0.978928
4,26336,1.061184,1.181181,0.936464,1.021152,0.071666,0.811645,0.989507
...,...,...,...,...,...,...,...,...
989,5683,0.982010,0.773861,0.785553,0.863654,0.077324,0.661153,0.992266
990,5682,0.841023,0.779998,0.747635,0.817129,0.079045,0.649416,0.991784
991,12772,1.246695,0.786818,1.030905,0.940693,0.165583,1.135882,0.968540
992,12771,0.742420,1.032760,0.889983,1.003829,0.077832,0.680220,0.990560


In [31]:
features = [f for f in df_default.columns if f not in ['cell_id', 'quality_avg']]
label = ['quality_avg']

In [120]:
X_train, X_test, y_train, y_test = train_test_split(df_default[features], df_default[label], test_size=0.7, random_state=3407)

In [155]:
X_train

NameError: name 'X_train' is not defined

In [122]:
y_test

,quality_avg
462,0.991632
105,0.987623
943,0.991124
19,0.991694
108,0.986945
...,...
94,0.914290
91,0.982617
312,0.992244
379,0.991741


## LinearRegression

In [123]:
model_lr = LinearRegression()

In [124]:
model_lr.fit(X_train, y_train)

LinearRegression()

In [125]:
model_lr.score(X_train, y_train)

0.7257335546707977

In [126]:
model_lr.score(X_test, y_test)

0.6542661250448853

In [127]:
model_lr.coef_

array([[-0.00534183, -0.00140214, -0.0023982 ,  0.0017004 , -0.04201931,
         0.00063019]])

In [128]:
model_lr.intercept_

array([0.99696222])

In [101]:
# dump(model_lr, osp.join(save_path, 'model_lr_69.joblib'))

['/home/rid/Projects/Study/Magister/Dyploma/Bachelor/data/generated/models/model_lr_69.joblib']

## PolynomialFeatures

In [144]:
transformer = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)

In [145]:
X_train_polynom = transformer.fit_transform(X_train)
X_test_polynom = transformer.transform(X_test)

In [146]:
model_pol_lr = LinearRegression().fit(X_train_polynom, y_train)

In [147]:
model_pol_lr.score(X_train_polynom, y_train)

0.8368586059929025

In [148]:
model_pol_lr.score(X_test_polynom, y_test)

0.7135514151520685

In [149]:
model_pol_lr.coef_

array([[-0.00698232, -0.01439924,  0.02981525, -0.01865063, -0.0338018 ,
         0.026771  , -0.00504854,  0.00516285, -0.00053048, -0.04011506,
         0.01278759, -0.01426853,  0.02428732,  0.07583166, -0.00449473,
        -0.00419851, -0.01667685, -0.02321615,  0.01082246,  0.00152197,
        -0.0793475 ]])

In [150]:
model_lr.intercept_

array([0.99696222])

In [102]:
# dump(model_pol_lr, osp.join(save_path, 'model_pol_lr_76.joblib'))

['/home/rid/Projects/Study/Magister/Dyploma/Bachelor/data/generated/models/model_pol_lr_76.joblib']

In [192]:
from sklearn.cross_decomposition import PLSRegression

In [205]:
reg = PLSRegression(n_components=6)

In [206]:
reg.fit(X_train, y_train.to_numpy().ravel())

PLSRegression(n_components=6)

In [207]:
reg.score(X_train, y_train.to_numpy().ravel())

0.7257335546707978

In [208]:
reg.score(X_test, y_test.to_numpy().ravel())

0.6542661250448858

In [170]:
reg.coef_

NameError: name 'reg' is not defined